# Instalamos los paquetes necesarios e importamos las librerías que vamos a utilizar

In [ ]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install camelot-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install ghostscript

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt install ghostscript python3-tk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ghostscript is already the newest version (9.26~dfsg+0-0ubuntu0.18.04.16).
python3-tk is already the newest version (3.6.9-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib 
import string
import math

import PyPDF2
import camelot



# Definimos las funciones que vamos a utilizar posteriormente

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Definición de variables
## =======================================================
# columns1 =['Index','Title', 'Year', 'ISSN', 'Categories', 'Category_Description', 'Total_cites']
# columns2 =['Index','Title', 'Impact_factor', 'Number_of_journals_in_category', 'Article_influence', 'Category_ranking', 'Inmediacy_index']
# columns3= ['Index','Title', 'Cited_half_life', '5_year_impact_factor', 'Eigenfactor', 'Rank_by_impact_factor', 'Quartile_rank']

columns1 =['Index','Title', 'Year', 'ISSN',  'Category_Description']
columns2 =['Index','Title','Long_Title','Total_cites', 'Impact_factor', 'Number_of_journals_in_category']
columns3= ['Index','Title','Article_influence','Category_ranking','Inmediacy_index', 'Cited_half_life', '5_year_impact_factor'] 
columns4= ['Index','Title', 'Eigenfactor', 'Rank_by_impact_factor', 'Quartile_rank']



Lo que haremos a continuación será coger cada pdf con su ruta especificada en la variable **information**, y coger las variables startpage y endpage también definidas en information para determinar en qué página hay que parar y generar un pdf.

**IMPORTANTE:** esto se hace como paso previo a la creación de los CSV ya que si los pdf son muy grandes, se consume mucha RAM y se para la ejecución. Se ahí la necesidad de trocear los PDF generados en el Impact Factor JCR



In [ ]:
## https://stackoverflow.com/questions/51567750/extract-specific-pages-of-pdf-and-save-it-with-python

import PyPDF2
from PyPDF2 import PdfFileReader


def split_pdfs(information):

    '''
    A esta función le pasamos un vector con la información del pdf y las páginas en las que
    queremos dividirlo. Nos devuelve los PDFs devueltos en los rangos establecidos.
    '''
    for page in range(len(information)):
        pdf_writer = PyPDF2.PdfFileWriter()
        pdf_writer_2 = PyPDF2.PdfFileWriter()

        '''
        Lo que haremos a continuación será coger cada pdf con su ruta especificada en information,
        y coger las variables startpage y endpage también definidas en information para determinar
        en qué página hay que parar y generar un pdf.
        '''

        '''
        IMPORTANTE: esto se hace como paso previo a la creación de los CSV ya que si los pdf son muy grandes,
        se consume mucha RAM y se para la ejecución. Se ahí la necesidad de trocear los PDF generados en el
        Impact Factor JCR
        '''
        pdf_reader = PyPDF2.PdfFileReader(information[page][0]) # carga el documento en la ruta especificada en la variable information
        start_1 = information[page][1] # coge el comienzo del rango actual de pdf
        end_1 = int( ( information[page][2]-information[page][1] )/2 ) + information[page][1] # calculamos un tra
        start_2 = int( ( information[page][2]-information[page][1] )/2 ) + information[page][1] + 1
        end_2 = information[page][2] # coge el final del rango actual del pdf
        
        '''
        En las siguientes líneas iremos agregando las páginas 1 a 1 según los límites que hemos definido
        en cada startpage y endpage en la variable information. Así, si el PDF hay que trocearlo en 
        3 o 4 partes principales, ya que tiene 3 o 4 saltos de columnas respectivamente,
        pues cada parte se trocea en 2. Por ejemplo, si en el PDF original existen 3 saltos de 
        columnas, tendremos 6 PDFs.
        '''

        print('\nRead File', information[page][0])
        while start_1<=end_1:
            pdf_writer.addPage(pdf_reader.getPage(start_1 -1))
            start_1+=1

        comienzo_1 = information[page][1]
        output_filename = '{}_{}_page_{}.pdf'.format(
            information[page][0],comienzo_1, end_1)
        print('\nOutput filename: ',output_filename)

        with open(output_filename,'wb') as out:
            pdf_writer.write(out)
            print('pdf 1 exportado')


        while start_2<=end_2:
            pdf_writer_2.addPage(pdf_reader.getPage(start_2 - 1))
            start_2+=1
        comienzo_2 = int( ( information[page][2]-information[page][1] )/2 ) + information[page][1] + 1
        output_filename_2 = '{}_{}_page_{}.pdf'.format(
            information[page][0],comienzo_2, end_2)
        print('\nOutput filename: ',output_filename_2)
        
        with open(output_filename_2,'wb') as out:
            pdf_writer_2.write(out)
            print('pdf 2 exportado')

**En la siguiente celda, definimos lás páginas del PDF donde se producen los saltos en las columnas, esto es, que en el PDF, al tener varias columnas y no caber todas en una página, durante el primer tramo aparecen todas las filas para 3-4 columnas, una vez que terminan las filas, el PDF salta a las siguientes columnas. De ahí que previamente haya que estudiar el PDF y ver donde se producen estos saltos de columnas**

In [ ]:
############################################################
# Tramos para los PDFs que vamos a generar. Estos tramos se obtienen después 
# donde se producen los saltos de las columnas en el PDF original. En este caso,
# estos son los tramos para los PDFs que tienen 4 saltos de columnas
############################################################

# # PDFs 2020 Impact Factor
# startpage1=1; endpage1=770; startpage2=771; endpage2=1540; startpage3=1541; endpage3=2310; startpage4=2311; endpage4=3080;

# # PDFs 2019 Impact Factor 
# startpage5=1; endpage5=761; startpage6=762; endpage6=1522; startpage7=1523; endpage7=2283; startpage8=2284; endpage8=3044;

# # PDFs 2018 Impact Factor 
# startpage9=1; endpage9=558; startpage10=559; endpage10=1116; startpage11=1117; endpage11=1674; startpage12=1675; endpage12=2232;

# # PDFs 2017 Impact Factor 
# startpage13=1; endpage13=547; startpage14=548; endpage14=1094; startpage15=1095; endpage15=1641; startpage16=1642; endpage16=2188;

# # PDFs 2016 Impact Factor 
# startpage17=1; endpage17=712; startpage18=713; endpage18=1424; startpage19=1425; endpage19=2136; startpage20=2137; endpage20=2848;

# # PDFs 2015 Impact Factor 
# startpage21=1; endpage21=706; startpage22=707; endpage22=1412; startpage23=1413; endpage23=2118; startpage24=2119; endpage24=2824;

# # PDFs 2014 Impact Factor 
# startpage25=1; endpage25=691; startpage26=692; endpage26=1382; startpage27=1383; endpage27=2073; startpage28=2074; endpage28=2764;

# # PDFs 2013 Impact Factor
# startpage29=1; endpage29=679; startpage30=680; endpage30=1358; startpage31=1359; endpage31=2037; startpage32=2038; endpage32=2716;

# # PDFs 2012 Impact Factor 
# startpage33=1; endpage33=671; startpage34=672; endpage34=1342; startpage35=1343; endpage35=2013; startpage36=2014; endpage36=2684;

# # PDFs 2011 Impact Factor 
# startpage37=1; endpage37=501; startpage38=502; endpage38=1256; startpage39=1257; endpage39=1503; startpage40=1504; endpage40=2004;

# # PDFs 2010 Impact Factor 
# startpage41=1; endpage41=628; startpage42=629; endpage42=1256; startpage43=1257; endpage43=1884; startpage44=1885; endpage44=2512;

# # PDFs 2009 Impact Factor 
# startpage45=1; endpage45=567; startpage46=568; endpage46=1134; startpage47=1135; endpage47=1701; startpage48=1702; endpage48=2268;

# # PDFs 2008 Impact Factor 
# startpage49=1; endpage49=254; startpage50=255; endpage50=508; startpage51=509; endpage51=762; startpage52=763; endpage52=1016;

# # PDFs 2007 Impact Factor 
# startpage53=1; endpage53=319; startpage54=320; endpage54=638; startpage55=639; endpage55=957; startpage56=958; endpage56=1276;

# # PDFs 2006 Impact Factor 
# startpage57=1; endpage57=238; startpage58=239; endpage58=476; startpage59=477; endpage59=714; startpage60=715; endpage60=952;

# # PDFs 2005 Impact Factor 
# startpage61=1; endpage61=235; startpage62=236; endpage62=470; startpage63=471; endpage63=705; startpage64=706; endpage64=940;

# # PDFs 2004 Impact Factor 
# startpage65=1; endpage65=230; startpage66=231; endpage66=460; startpage67=461; endpage67=690; startpage68=691; endpage68=920;

# # PDFs 2002 Impact Factor 
# startpage69=1; endpage69=380; startpage70=381; endpage70=760; startpage71=761; endpage71=1140; startpage72=1141; endpage72=1520;

#==========================================================
#==========================================================
# Para los PDFs que SÍ tienen la columna del nombre de revista, estas serían las 
# páginas que hay que indicarle al script para separar el PDF original en esos tramos
#==========================================================
#==========================================================


## Indicamos la ruta del PDF que vamos a trocear en los rangos que correspondan

##################################
## Rutas de acceso para llegar los PDFs de 2020-2010
##################################

# filename1_route= ['/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020_nueva_columna/Impact_factor_2020_columna_titulo.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2019_con_columna_nombre_de_revista/impact_factor_2019_con_nombre_revista.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2018_con_columna_nombre_de_revista/impact_factor_2018_con_nombre_revista.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2017_con_columna_nombre_de_revista/impact_factor_2017_con_nombre_revista.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2016_con_columna_nombre_de_revista/impact_factor_2016_con_nombre_revista.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2015_con_columna_nombre_de_revista/impact_factor_2015_con_nombre_revista.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2014_con_columna_nombre_de_revista/impact_factor_2014_con_nombre_revista.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2013_con_columna_nombre_de_revista/impact_factor_2013_con_nombre_revista.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2012_con_columna_nombre_de_revista/impact_factor_2012_con_nombre_revista.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2011_con_columna_nombre_de_revista/impact_factor_2011_con_nombre_revista.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2010_con_columna_nombre_de_revista/impact_factor_2010_con_nombre_revista.pdf',
#                   ]

# year = ['2020','2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010']

##################################
## Rutas de acceso para llegar a los PDFs de 2009-2002
##################################

# filename1_route = ['/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2009_con_columna_nombre_de_revista/impact_factor_2009_con_nombre_revista.pdf',
#                    '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2008_con_columna_nombre_de_revista/impact_factor_2008_con_nombre_revista.pdf',
#                    '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2007_con_columna_nombre_de_revista/impact_factor_2007_con_nombre_revista.pdf',
#                    '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2006_con_columna_nombre_de_revista/impact_factor_2006_con_nombre_revista.pdf',
#                    '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2005_con_columna_nombre_de_revista/impact_factor_2005_con_nombre_revista.pdf',
#                    '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2004_con_columna_nombre_de_revista/impact_factor_2004_con_nombre_revista.pdf',
#                    '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2002_con_columna_nombre_de_revista/impact_factor_2002_con_nombre_revista.pdf'                  
#                    ]

# year = ['2009', '2008', '2007','2006','2005', '2004', '2002']



##################################
#PARA LOS AÑOS 2020-2010
##################################

# information = [
#                [(filename1_route[0],startpage1,endpage1), 
#                (filename1_route[0], startpage2, endpage2),
#                (filename1_route[0], startpage3, endpage3),
#                (filename1_route[0], startpage4, endpage4)],
#                [(filename1_route[1],startpage5,endpage5), 
#                (filename1_route[1], startpage6, endpage6),
#                (filename1_route[1], startpage7, endpage7),
#                (filename1_route[1], startpage8, endpage8)],
#                [(filename1_route[2],startpage9,endpage9), 
#                (filename1_route[2], startpage10, endpage10),
#                (filename1_route[2], startpage11, endpage11),
#                (filename1_route[2], startpage12, endpage12)],
#                [(filename1_route[3],startpage13,endpage13), 
#                (filename1_route[3], startpage14, endpage14),
#                (filename1_route[3], startpage15, endpage15),
#                (filename1_route[3], startpage16, endpage16)],
#                [(filename1_route[4],startpage17,endpage17), 
#                (filename1_route[4], startpage18, endpage18),
#                (filename1_route[4], startpage19, endpage19),
#                (filename1_route[4], startpage20, endpage20)],
#                [(filename1_route[5],startpage21,endpage21), 
#                (filename1_route[5], startpage22, endpage22),
#                (filename1_route[5], startpage23, endpage23),
#                (filename1_route[5], startpage24, endpage24)],
#                [(filename1_route[6],startpage25,endpage25), 
#                (filename1_route[6], startpage26, endpage26),
#                (filename1_route[6], startpage27, endpage27),
#                (filename1_route[6], startpage28, endpage28)],
#                [(filename1_route[7],startpage29,endpage29), 
#                (filename1_route[7], startpage30, endpage30),
#                (filename1_route[7], startpage31, endpage31),
#                (filename1_route[7], startpage32, endpage32)],
#                [(filename1_route[8],startpage33,endpage33), 
#                (filename1_route[8], startpage34, endpage34),
#                (filename1_route[8], startpage35, endpage35),
#                (filename1_route[8], startpage36, endpage36)],
#                [(filename1_route[9],startpage37,endpage37), 
#                (filename1_route[9], startpage38, endpage38),
#                (filename1_route[9], startpage39, endpage39),
#                (filename1_route[9], startpage40, endpage40)],
#                [(filename1_route[10],startpage41,endpage41), 
#                (filename1_route[10], startpage42, endpage42),
#                (filename1_route[10], startpage43, endpage43),
#                (filename1_route[10], startpage44, endpage44)]]

##################################
#PARA LOS AÑOS 2009-2002 (SIN 2003)
##################################

# information = [
#                [(filename1_route[0],startpage45,endpage45), 
#                (filename1_route[0], startpage46, endpage46),
#                (filename1_route[0], startpage47, endpage47),
#                (filename1_route[0], startpage48, endpage48)],
#                [(filename1_route[1],startpage49,endpage49), 
#                (filename1_route[1], startpage50, endpage50),
#                (filename1_route[1], startpage51, endpage51),
#                (filename1_route[1], startpage52, endpage52)],
#                [(filename1_route[2],startpage53,endpage53), 
#                (filename1_route[2], startpage54, endpage54),
#                (filename1_route[2], startpage55, endpage55),
#                (filename1_route[2], startpage56, endpage56)],
#                [(filename1_route[3],startpage57,endpage57), 
#                (filename1_route[3], startpage58, endpage58),
#                (filename1_route[3], startpage59, endpage59),
#                (filename1_route[3], startpage60, endpage60)],
#                [(filename1_route[4],startpage61,endpage61), 
#                (filename1_route[4], startpage62, endpage62),
#                (filename1_route[4], startpage63, endpage63),
#                (filename1_route[4], startpage64, endpage64)],
#                [(filename1_route[5],startpage65,endpage65), 
#                (filename1_route[5], startpage66, endpage66),
#                (filename1_route[5], startpage67, endpage67),
#                (filename1_route[5], startpage68, endpage68)],
#                [(filename1_route[6],startpage69,endpage69), 
#                (filename1_route[6], startpage70, endpage70),
#                (filename1_route[6], startpage71, endpage71),
#                (filename1_route[6], startpage72, endpage72)]

#               ]

#==========================================================
#==========================================================
# Para los PDFs que NO tienen la columna del nombre de revista, estas serían las 
# páginas que hay que indicarle al script para separar el PDF original en esos tramos.
# La diferencia con las líneas de arriba aparte de los nombres de los archivos es
# que cada PDF tiene diferentes saltos de columnas, por lo que sus tramos de páginas 
# serán diferentes, en este caso, en las siguientes líneas se tienen 3 saltos de columnas.
#==========================================================
#==========================================================

##################################
#PARA LOS AÑOS 2020-2010
##################################

# year = ['2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010']

# filename1_route= ['/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2019/Impact_factor_2019.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2018/Impact_factor_2018.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2017/Impact_factor_2017.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2016/Impact_factor_2016.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2015/Impact_factor_2015.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2014/Impact_factor_2014.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2013/Impact_factor_2013.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2012/Impact_factor_2012.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2011/Impact_factor_2011.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2010/Impact_factor_2010.pdf']

##################################
#PARA LOS AÑOS 2009-2002 (SIN 2003)
##################################

# filename1_route= ['/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2009/Impact_factor_2009.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2008/Impact_factor_2008.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2007/Impact_factor_2007.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2006/Impact_factor_2006.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2005/Impact_factor_2005.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2004/Impact_factor_2004.pdf',
#                   '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2002/Impact_factor_2002.pdf'
#                   ]

#year = ['2009', '2008', '2007','2006','2005', '2004', '2002']

# # PDFs 2019 Impact Factor 2019: 1-761, 762-1522, 1523-2283
# startpage1=1; endpage1=761; startpage2=762; endpage2=1522; startpage3=1523; endpage3=2283;

# # PDFs 2018 Impact Factor 2018: 1-558, 559-1116, 1117-1674
# startpage4=1; endpage4=558; startpage5=559; endpage5=1116; startpage6=1117; endpage6=1674;

# # PDFs 2017 Impact Factor 2017: 1-724, 725-1448, 1449-2172
# startpage7=1; endpage7=724; startpage8=725; endpage8=1448; startpage9=1449; endpage9=2172;

# # PDFs 2016 Impact Factor 2016: 1-539, 540-1078, 1079-1617
# startpage10=1; endpage10=539; startpage11=540; endpage11=1078; startpage12=1079; endpage12=1617;

# # PDFs 2015 Impact Factor 2015: 1-534, 535-1068, 1069-1602
# startpage13=1; endpage13=534; startpage14=535; endpage14=1068; startpage15=1069; endpage15=1602;

# # PDFs 2014 Impact Factor 2014: 1-691, 692-1382, 1383-2073
# startpage16=1; endpage16=691; startpage17=692; endpage17=1382; startpage18=1383; endpage18=2073;

# # PDFs 2013 Impact Factor 2013: 1-679, 680-1358, 1359-2037
# startpage19=1; endpage19=679; startpage20=680; endpage20=1358; startpage21=1359; endpage21=2037;

# # PDFs 2012 Impact Factor 2012: 1-671, 672-1342, 1343-2013
# startpage22=1; endpage22=671; startpage23=672; endpage23=1342; startpage24=1343; endpage24=2013;

# # PDFs 2011 Impact Factor 2011: 1-657, 658-1314, 1315-1971
# startpage25=1; endpage25=657; startpage26=658; endpage26=1314; startpage27=1315; endpage27=1971;

# # PDFs 2010 Impact Factor 2010: 1-484, 485-968, 969-1452
# startpage28=1; endpage28=484; startpage29=485; endpage29=968; startpage30=969; endpage30=1452;

# # PDFs 2009 Impact Factor 2009: 
# startpage31=1; endpage31=567; startpage32=568; endpage32=1134; startpage33=1135; endpage33=1701;

# # PDFs 2008 Impact Factor 2008: 
# startpage34=1; endpage34=316; startpage35=317; endpage35=632; startpage36=633; endpage36=948;

# # PDFs 2007 Impact Factor 2007:
# startpage37=1; endpage37=319; startpage38=320; endpage38=638; startpage39=639; endpage39=957;

# # PDFs 2006 Impact Factor 2006:
# startpage40=1; endpage40=306; startpage41=307; endpage41=612; startpage42=613; endpage42=918;

# # PDFs 2005 Impact Factor 2005:
# startpage43=1; endpage43=302; startpage44=303; endpage44=604; startpage45=605; endpage45=906;

# # PDFs 2004 Impact Factor 2004: 
# startpage46=1; endpage46=296; startpage47=297; endpage47=592; startpage48=593; endpage48=888;

# # # PDFs 2003 Impact Factor 2003: 1-484, 485-968, 969-1452 //// No hay documento
# # startpage28=1; endpage28=484; startpage29=485; endpage29=968; startpage30=969; endpage30=1452;

# # PDFs 2002 Impact Factor 2002: 
# startpage49=1; endpage49=314; startpage50=315; endpage50=628; startpage51=629; endpage51=942;

# PARA LOS AÑOS DE 2020-2010
# information = [
#                [(filename1_route[0],startpage1,endpage1), 
#                (filename1_route[0], startpage2, endpage2),
#                (filename1_route[0], startpage3, endpage3)],
#                [(filename1_route[1],startpage4,endpage4), 
#                (filename1_route[1], startpage5, endpage5),
#                (filename1_route[1], startpage6, endpage6)],
#                [(filename1_route[2],startpage7,endpage7), 
#                (filename1_route[2], startpage8, endpage8),
#                (filename1_route[2], startpage9, endpage9)],
#                [(filename1_route[3],startpage10,endpage10), 
#                (filename1_route[3], startpage11, endpage11),
#                (filename1_route[3], startpage12, endpage12)],
#                [(filename1_route[4],startpage13,endpage13), 
#                (filename1_route[4], startpage14, endpage14),
#                (filename1_route[4], startpage15, endpage15)],
#                [(filename1_route[5],startpage16,endpage16), 
#                (filename1_route[5], startpage17, endpage17),
#                (filename1_route[5], startpage18, endpage18)],
#                [(filename1_route[6],startpage19,endpage19), 
#                (filename1_route[6], startpage20, endpage20),
#                (filename1_route[6], startpage21, endpage21)],
#                [(filename1_route[7],startpage22,endpage22), 
#                (filename1_route[7], startpage23, endpage23),
#                (filename1_route[7], startpage24, endpage24)],
#                [(filename1_route[8],startpage25,endpage25), 
#                (filename1_route[8], startpage26, endpage26),
#                (filename1_route[8], startpage27, endpage27)],
#                [(filename1_route[9],startpage28,endpage28), 
#                (filename1_route[9], startpage29, endpage29),
#                (filename1_route[9], startpage30, endpage30)]
#                ]

#PARA LOS AÑOS 2009-2002 (SIN 2003)
# information = [
#                [(filename1_route[0],startpage31,endpage31), 
#                (filename1_route[0], startpage32, endpage32),
#                (filename1_route[0], startpage33, endpage33)],
#                [(filename1_route[1],startpage34,endpage34), 
#                (filename1_route[1], startpage35, endpage35),
#                (filename1_route[1], startpage36, endpage36)],
#                [(filename1_route[2],startpage37,endpage37), 
#                (filename1_route[2], startpage38, endpage38),
#                (filename1_route[2], startpage39, endpage39)],
#                [(filename1_route[3],startpage40,endpage40), 
#                (filename1_route[3], startpage41, endpage41),
#                (filename1_route[3], startpage42, endpage42)],
#                [(filename1_route[4],startpage43,endpage43), 
#                (filename1_route[4], startpage44, endpage44),
#                (filename1_route[4], startpage45, endpage45)],
#                [(filename1_route[5],startpage46,endpage46), 
#                (filename1_route[5], startpage47, endpage47),
#                (filename1_route[5], startpage48, endpage48)],
#                [(filename1_route[6],startpage49,endpage49), 
#                (filename1_route[6], startpage50, endpage50),
#                (filename1_route[6], startpage51, endpage51)]
#                ]


#==========================================================
#==========================================================
# Realizamos la división de los PDFs en los rangos establecidos,
# Se guardarán utilizano la misma ruta del pdf que se esté usando 
# en ese momento 
#==========================================================
#==========================================================



################################################
# Para los siguientes:
################################################
filename_route =['/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2019/Impact_factor_2019.pdf',
                 '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2019/Impact_factor_2019.pdf']

# startpage1 =  ; endpage1 = ;startpage2 =  ; endpage2 = ;startpage3 =  ; endpage3 = ;startpage4 =  ; endpage4 = ;
startpage1=1; endpage1=761; startpage2=762; endpage2=1522; startpage3=1523; endpage3=2283;
year=['2019','2019']
information = [
               [(filename_route[0], startpage1, endpage1),
               (filename_route[0], startpage2, endpage2),
               (filename_route[0], startpage3, endpage3)],
               [(filename_route[0], startpage1, endpage1),
               (filename_route[0], startpage2, endpage2),
               (filename_route[0], startpage3, endpage3)]
               ]



En la siguiente celda ejecutamos lo que hemos definido previamente en la anterior celda.

In [ ]:
# split_pdfs(information)
for i in np.arange(len(year)):
  print(i)
  split_pdfs(information[i])

0

Read File /content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2019/Impact_factor_2019.pdf

Output filename:  /content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2019/Impact_factor_2019.pdf_1_page_381.pdf
pdf 1 exportado

Output filename:  /content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2019/Impact_factor_2019.pdf_382_page_761.pdf
pdf 2 exportado

Read File /content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2019/Impact_factor_2019.pdf

Output filename:  /content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2019/Impact_factor_2019.pdf_762_page_1142.pdf
pdf 1 exportado

Output filename:  /content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2019/Impact_factor_2019.pdf_1143_page_1522.pdf
pdf 2 exportado

Read File /content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2019/Impact_factor_2019.pdf

Output filename:  /c

# A continuación definimos las funciones que vamos a utilizar. 
**NOTA:** Hay que ejecutar esta celda para poder realizar las transformaciones

In [ ]:
def load_pdf_into_tables(filename_route):

  '''
  Pasamos la ruta del archivo PDF.
  Nos devuelve tabla leída de cada página, en una variable.
  Cada tabla es accesible con índices [i], e.g.: table[0]
  '''
  table = camelot.read_pdf(filename_route, pages='all')
  print("Total tables extracted:", table.n)
  
  return table

def concatenated_df(df1, df2):
    '''
    Concatenamos los dataframes
    Devuelve el dataframe concatenado
    '''
    df = pd.concat([df1,df2]).reset_index(drop= True)
    print('Dataframe concatenated!')
    return df

def save_df_to_csv(df, filename_route_out='', year='', i=0):
    '''
    Pasamos el dataframe con las mismas columnas ya unido
    Guardamos el CSV en el directorio escogido, pasándole además el año y el 
    índice indicado.
    '''
    df.to_csv(filename_route_out  + '/Impact_factor_' + year + '_'+ str(i) + '.csv', sep = ';',index = False) 
    # df = pd.read_csv(filename_route_out  + '/Impact_factor_' + year + '_'+ str(i) + '.csv', sep = ';')
    # df = df.drop('Unnamed: 0', axis = 1)

    #Arreglamos la columna índice y título
    # df = extract_index_text(999, df_1_1)
    # df.to_csv(filename_route_out  + '/Impact_factor_' + year + '_'+ str(i) + '.csv', sep = ';', index = False)
    print('CSV saved!')

def read_and_format_columns_from_tables(tables, columns = []):

    '''
    La función lee las tabla previamente cargada con la función load_pdf_into_tables.
    Elimina la primera fila, cambiamos el nombre a  las columnas.
    Nos devuelve la tabla en cuestión, en dataframe.

    '''
    #Leemos la tabla que previamente hemos cargado con la librería Camelot
    df = pd.DataFrame(tables.df)
    # Eliminamos la primera fila, que contiene el nombre de las columnas
    df = df.drop(0, axis = 0)
    #cambiamos el nombre de las columnas a las que hemos establecido
    df.columns = columns
#     test.head()
    #devolvemos el dataframe formateado
    return df

def df_joined_rows(table):
    '''
  1. Pasamos las tablas cargadas previamente con la librería camelot
  2. Usamos la función read_and_format_columns_from_tables para convertir esas tablas a Dataframe
  con las columnas indicadas
  3. Concatenamos todos esos Dataframes en un dataframe final
    '''
    #iniciamos la variable columns
    columns = table[0].df.iloc[0,:]
    #concatenamos todas los dataframes que nos va devolviendo las función read_and_format_columns_from_tables
    for i in np.arange(0,table.n):
      if i == 0:
          # inciamos el dataframe al cual se le van a concatenar todos los df's que hemos convertido
          # previamente 
          df_rows_joined = pd.concat([read_and_format_columns_from_tables(table[i], columns), read_and_format_columns_from_tables(table[i+1], columns)])
      else:
        try:
          # indicamos el +1 dentro del try except para que concatene la siguiente tabla
          # a la actual, cuando se produzca el out of bound, siga ejecutándose y 
          # termine
          df_rows_joined = pd.concat([df_rows_joined, read_and_format_columns_from_tables(table[i+1], columns)])
        except:
          pass

    # Devuelve en un dataframe todas las tablas leídas del pdf y concatenadas una
    # debajo de otra
    return df_rows_joined.reset_index(drop= True)

def read_csv(csv_filename_route):
  '''
  Cargamos el dataframe
  '''
  df = pd.read_csv(csv_filename_route, sep=';')
  # df = df.drop('Unnamed: 0', axis = 1)
  return df


################################################
# Versión actualizada - En el PDF con los nombres de las revistas no existe la columna 'Categories'
# por lo que esta función no tienen cabida. Sin embargo en los PDF donde no estaban incluidos
# los nombres, sí aparece esta colunma, por lo que producía un solapamiento entre 
# 'CATEGORY_DESCRIPTION' y 'Categories' 
################################################

# Le pasamos, además del dataframe, el parámetro categories, que serán todos los valores posibles de las
# categorías. Esto lo obtenemos con la función unique().
def replace_nan_values_in_category_description(df, categories = []):
  df_1_copy=df.copy()
  '''
  En esta función recorreremos todas las categorías extraídas previamente.
  En aquellos valores de la columna CATEGORY_DESCRIPTION que sean nulos, extraemos 
  el índice para saber en qué posición esta columna es nula.
  Mediante el IF evaluamos si la categoría que estamos evaluando en ese momento se encuentra en la columna Categories.
  Si se encuentra, actualizamos el valor de CATEGORY_DESCRIPTION con la categoría
  que estamos evaluando en ese momento y por otro lado, la eliminamos de la columna Categories.
  De esta manera CATEGORY_DESCRIPTION no será nulo y tendrá su valor correcto  Categories
  ya no tendrá el valor solapado.
  '''
  for i in np.arange(len(categories)):
    for index in df_1_copy[df_1_copy['CATEGORY_DESCRIPTION'].isnull()].index:
      if categories[i] in df_1_copy['Categories'][index]:
        # print('\nFound!')
        # print('Category:', categories[i])
        df_1_copy.iloc[index,5]= categories[i]
        # df_1_copy.iloc[index,4] = categories[i]
        # print('Updated value:', df_1_copy['CATEGORY_DESCRIPTION'][index] )
        # print('Index_text', index)
        df_1_copy.iloc[index,4]= df_1_copy['Categories'][index].replace(categories[i],'')

  return df_1_copy

def extract_index_text(starter_row_index, df_test):
  # https://stackoverflow.com/questions/12851791/removing-numbers-from-string

  '''
  Esta función nace con la finalidad de corregir un problema que afectaba a la columna 
  del índice. Al llegar al elemento 999, se solapaban esta columna con la columna 'Title20'.
  Por lo tanto la idea es extraer de Index el texto y actualizar Title20(que será NaN) con el nombre
  de la revista, y a su vez Index se actualice con sólo el valor del índice (no índice+title20)
  '''

  ## NOTA: para corregir otras columnas a las que les pasa lo mismo, se debe
  ## implementar un IF que discrimine por valores nulo esa columna. 

  for j in np.arange(starter_row_index,df_test.shape[0]): # indicamos en qué indice empieza a hacer la transformación y cual es el final

    # recorremos la columna Index, evaluando si es Na cada valor j
    if pd.isna(df_test[df_test.columns[0]][j]) != True:
      # Cuando la columna Index no es Nan, se extraen el índice y el texto de la columna
      # Index y se actualizan los datos tanto en Title20 como en Index con el valor 
      # correcto

      extracted_text = ''.join([i for i in df_test[df_test.columns[0]][j]  if not i.isdigit()])
      
      removed_text = ''.join([i for i in df_test[df_test.columns[0]][j] if not i.isalpha()]).replace('+','').replace('-','').strip()
      
      if extracted_text != '':
        
        df_test.iloc[j,1] = extracted_text[1:]# [1:] es para quitarle el espacio inicial
        df_test.iloc[j,0] = removed_text
    else:
      # Cuando la columna Index es Nan, el valor del índice y el nombre corto de la revista
      # se extraen de la columna Title20. 

      # print('Original data ',df_test[df_test.columns[1]][j])
      removed_text = ''.join([i for i in df_test[df_test.columns[1]][j] if not i.isalpha()]).replace('+','').replace('-','').strip() # eliminar '-' y eliminar espacios

      extracted_text = ''.join([i for i in df_test[df_test.columns[1]][j] if not i.isdigit()])
      # print(j)
      # print('Extracted_text',extracted_text)
      # print('Removed text',removed_text)
      
      df_test.iloc[j,1] = extracted_text[1:]
      # print('Updated value', df_test[df_test.columns[1]][j], '\n')
      df_test.iloc[j,0]= removed_text 
  
  return df_test

############################################
# Versión para los 3 dataframes generados a partir de los PDFs 
# sin nombre de revista. Concatena los 3 dataframes en uno final
############################################

# def create_final_csv(df_1, df_2,df_3):
#   # https://stackoverflow.com/questions/14984119/python-pandas-remove-duplicate-columns
#   # https://www.statology.org/pandas-merge-on-index/
#   # https://www.w3schools.com/sql/sql_join.asp

#   #concatenamos las 3 tablas
#   df = pd.concat([df_1, df_2, df_3], axis = 1)
#   # eliminamos las columnas duplicadas
#   df = df.loc[:,~df.columns.duplicated()]

#   return df

############################################
# Versión para los 4 dataframes generados a partir de los PDFs 
# con nombre de revista. Concatena los 4 dataframes en un final.
############################################
def create_final_csv(df_1, df_2,df_3, df_4):
  # https://stackoverflow.com/questions/14984119/python-pandas-remove-duplicate-columns
  # https://www.statology.org/pandas-merge-on-index/
  # https://www.w3schools.com/sql/sql_join.asp

  #concatenamos las 3 tablas
  df = pd.concat([df_1, df_2, df_3,df_4], axis = 1)
  # eliminamos las columnas duplicadas
  df = df.loc[:,~df.columns.duplicated()]

  return df



######################################################
# Función para limpiar la columna del nombre de la revista de NaN
######################################################


def fixing_publisher_column(df_final):
  df_final_copy = df_final.copy()
  count=0

  print('##### paso 1 #####')
  '''
  En este primer paso, rellenamos los nan de la columna Publisher con los nombres de las
  revistas que se han incrustado en la columna TOT_CITES.
  '''
  for j in np.arange(0,len(df_final_copy)):
    if type(df_final_copy['Publisher'].iloc[j]) == str:
      ### Comprobamos que elemento j de la columna Publisher es de tipo string
      pass
    elif (type(df_final_copy['Publisher'].iloc[j]) == float) & (type(df_final_copy['TOT_CITES'].iloc[j]) == str) & (type(df_final_copy['IMPACT_FACTOR'].iloc[j]) == float):
      '''
      Si se cumple que el elemento j de la columna Publisher es float(por tanto NaN)
      y además se cumplen el resto de condiciones, implica que el nombre de la revista está en la columna TOT_CITIES 
      (esto lo sabemos previo estudio de los NaN en las columnas Publisher, TOT_CITES e IMPACT_FACTOR),
      entonces sustituímos el valor NaN de Publisher por el detectado en la columna
      TOT_CITES
      '''
      # print('\n### Primera transformación ###', j)
      print('Nan element (PUBLISHER): ', df_final_copy['Publisher'].iloc[j])
      df_final_copy['Publisher'].iloc[j] = df_final_copy['TOT_CITES'].iloc[j]
      print('Updated element: ', df_final_copy['Publisher'].iloc[j])

    elif (type(df_final_copy['Publisher'].iloc[j]) == float) & (type(df_final_copy['TOT_CITES'].iloc[j]) == str) & (type(df_final_copy['IMPACT_FACTOR'].iloc[j]) == str):
      '''
      Si se cumple que el elemento j de la columna Publisher es float(por tanto NaN)
      y además se cumplen el resto de condiciones (IMPACT_FACTOR es string en vez de float como en la anterior)
      , implica que el nombre de la revista TAMBIÉN está en la columna TOT_CITIES 
      (esto lo sabemos previo estudio de los NaN en las columnas Publisher, TOT_CITES e IMPACT_FACTOR),
      entonces sustituímos el valor NaN de Publisher por el detectado en la columna
      TOT_CITES
      '''
      # print('\n### Primera transformación ###', j)
      print('Nan element (PUBLISHER): ', df_final_copy['Publisher'].iloc[j])
      df_final_copy['Publisher'].iloc[j] = df_final_copy['TOT_CITES'].iloc[j]
      print('Updated element: ', df_final_copy['Publisher'].iloc[j])
  j = 0;
  print('\n\n##### paso 2 #####\n')
  for j in np.arange(0,len(df_final_copy)):
    '''
    Una vez que ya no tenemos NaN en la columna de Publisher, comprobamos si es float (NaN)
    Comprobamos si la última posición del elemento j es un dígito o una letra
    Si es un dígito, contamos cuantos dígitos tiene. Una vez que tenemos la cuenta de los dígitos,
    eliminamos justo esa cantidad de dígitos del final del elemento j, limpiando así
    el nombre de la revista del total de citas que se ha solapado.
    '''
    if type(df_final_copy['Publisher'].iloc[j]) != float:
      ## Aunque esta comprobación se puede evitar ya que previamente se han limpiado los NaN,
      ## lo heredamos como se concibió al principio, es decir sin el paso 1.
      ## Comprobamos si el elemento no es float o no, ya que puede haber algunos nan
      

      if df_final_copy['Publisher'].iloc[j][-1].isdigit() == True:
        # Si no es float, comprobamos si el último elemento de la cadena strng es un numero
        

        for i in np.arange(1,len(df_final_copy['Publisher'].iloc[j])):
          ### si el ultimo elemento es un numero, iniciamos un for para recorrer esa cadena desde el final al principio,
          ###  ya que habremos comprobado que se ha solapado el total de citas con el nombre de la revista
          
          
          if (df_final_copy['Publisher'].iloc[j][-i].isdigit() == True):
            ### comprobamos todos los dígitos desde el final y aumentamos la cuenta
            count+=1
            # print('\ncuenta1', count)
          else:
            ### una vez que no sea dígito, entramos aquí y eliminamos los -i elementos
            ### utilzando el contador cuenta, que nos indica cuantos dígitos hay
            print('not updated publisher', df_final_copy['Publisher'].iloc[j])
            df_final_copy['Publisher'].iloc[j] = df_final_copy['Publisher'].iloc[j][:-i+1]
            # print('\ncuenta2', count)

            print('updated publisher', df_final_copy['Publisher'].iloc[j])
            count = 0       
            break 

      elif df_final_copy['Publisher'].iloc[j][-1].isdigit() != True:
        '''
        Si el último elemento no es dígito, quiere decir que el total de citas
        no se ha solapado en la columna de Publisher, luego vamos al siguiente.
        '''
        pass

    elif type(df_final_copy['Publisher'].iloc[j]) == float:
      '''
      Si el elemento fuera un float(nan), pasaríamos al siguiente elemento j
      '''
      pass
      
  
  print(df_final_copy.isnull().sum()/df_final_copy.shape[0])

  return df_final_copy

**Con el siguiente script extraemos los csv utilizando todas las funciones previamente definidas, y las metemos en un bucle FOR para automatizarlo**. Para ello previamente hay que definir los años, las rutas a los PDFs y las rutas de las carpetas de salida. Todos estos datos deben coincidir en índice para que el bucle FOR coja los datos correctos en cada iteración.

In [ ]:
## Este script nos genera los csv de cada conjunto de pdf

# ========================================================================
# # Años 2019-2010
# ========================================================================

# year = ['2019','2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010']
# 2020 no está porque ya lo había hecho anteriormente como prueba

# files_route= [
                # [
                # ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2019_con_columna_nombre_de_revista/impact_factor_2019_con_nombre_revista.pdf_1_page_381.pdf',
                # '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2019_con_columna_nombre_de_revista/impact_factor_2019_con_nombre_revista.pdf_382_page_761.pdf'),
                # ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2019_con_columna_nombre_de_revista/impact_factor_2019_con_nombre_revista.pdf_762_page_1142.pdf',
                # '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2019_con_columna_nombre_de_revista/impact_factor_2019_con_nombre_revista.pdf_1143_page_1522.pdf'),
                # ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2019_con_columna_nombre_de_revista/impact_factor_2019_con_nombre_revista.pdf_1523_page_1903.pdf',
                # '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2019_con_columna_nombre_de_revista/impact_factor_2019_con_nombre_revista.pdf_1904_page_2283.pdf'),
                # ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2019_con_columna_nombre_de_revista/impact_factor_2019_con_nombre_revista.pdf_2284_page_2664.pdf',
                # '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2019_con_columna_nombre_de_revista/impact_factor_2019_con_nombre_revista.pdf_2665_page_3044.pdf')],
#               [
#                ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2018_con_columna_nombre_de_revista/impact_factor_2018_con_nombre_revista.pdf_1_page_279.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2018_con_columna_nombre_de_revista/impact_factor_2018_con_nombre_revista.pdf_280_page_558.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2018_con_columna_nombre_de_revista/impact_factor_2018_con_nombre_revista.pdf_559_page_837.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2018_con_columna_nombre_de_revista/impact_factor_2018_con_nombre_revista.pdf_838_page_1116.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2018_con_columna_nombre_de_revista/impact_factor_2018_con_nombre_revista.pdf_1117_page_1395.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2018_con_columna_nombre_de_revista/impact_factor_2018_con_nombre_revista.pdf_1396_page_1674.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2018_con_columna_nombre_de_revista/impact_factor_2018_con_nombre_revista.pdf_1675_page_1953.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2018_con_columna_nombre_de_revista/impact_factor_2018_con_nombre_revista.pdf_1954_page_2232.pdf')],
#               [
#                ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2017_con_columna_nombre_de_revista/impact_factor_2017_con_nombre_revista.pdf_1_page_274.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2017_con_columna_nombre_de_revista/impact_factor_2017_con_nombre_revista.pdf_275_page_547.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2017_con_columna_nombre_de_revista/impact_factor_2017_con_nombre_revista.pdf_548_page_821.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2017_con_columna_nombre_de_revista/impact_factor_2017_con_nombre_revista.pdf_822_page_1094.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2017_con_columna_nombre_de_revista/impact_factor_2017_con_nombre_revista.pdf_1095_page_1368.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2017_con_columna_nombre_de_revista/impact_factor_2017_con_nombre_revista.pdf_1369_page_1641.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2017_con_columna_nombre_de_revista/impact_factor_2017_con_nombre_revista.pdf_1642_page_1915.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2017_con_columna_nombre_de_revista/impact_factor_2017_con_nombre_revista.pdf_1916_page_2188.pdf')],
#               [
#                ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2016_con_columna_nombre_de_revista/impact_factor_2016_con_nombre_revista.pdf_1_page_356.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2016_con_columna_nombre_de_revista/impact_factor_2016_con_nombre_revista.pdf_357_page_712.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2016_con_columna_nombre_de_revista/impact_factor_2016_con_nombre_revista.pdf_713_page_1068.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2016_con_columna_nombre_de_revista/impact_factor_2016_con_nombre_revista.pdf_1069_page_1424.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2016_con_columna_nombre_de_revista/impact_factor_2016_con_nombre_revista.pdf_1425_page_1780.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2016_con_columna_nombre_de_revista/impact_factor_2016_con_nombre_revista.pdf_1781_page_2136.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2016_con_columna_nombre_de_revista/impact_factor_2016_con_nombre_revista.pdf_2137_page_2492.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2016_con_columna_nombre_de_revista/impact_factor_2016_con_nombre_revista.pdf_2493_page_2848.pdf')],
#               [
#                ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2015_con_columna_nombre_de_revista/impact_factor_2015_con_nombre_revista.pdf_1_page_353.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2015_con_columna_nombre_de_revista/impact_factor_2015_con_nombre_revista.pdf_354_page_706.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2015_con_columna_nombre_de_revista/impact_factor_2015_con_nombre_revista.pdf_707_page_1059.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2015_con_columna_nombre_de_revista/impact_factor_2015_con_nombre_revista.pdf_1060_page_1412.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2015_con_columna_nombre_de_revista/impact_factor_2015_con_nombre_revista.pdf_1413_page_1765.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2015_con_columna_nombre_de_revista/impact_factor_2015_con_nombre_revista.pdf_1766_page_2118.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2015_con_columna_nombre_de_revista/impact_factor_2015_con_nombre_revista.pdf_2119_page_2471.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2015_con_columna_nombre_de_revista/impact_factor_2015_con_nombre_revista.pdf_2472_page_2824.pdf')],
#               [
#                ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2014_con_columna_nombre_de_revista/impact_factor_2014_con_nombre_revista.pdf_1_page_346.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2014_con_columna_nombre_de_revista/impact_factor_2014_con_nombre_revista.pdf_347_page_691.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2014_con_columna_nombre_de_revista/impact_factor_2014_con_nombre_revista.pdf_692_page_1037.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2014_con_columna_nombre_de_revista/impact_factor_2014_con_nombre_revista.pdf_1038_page_1382.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2014_con_columna_nombre_de_revista/impact_factor_2014_con_nombre_revista.pdf_1383_page_1728.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2014_con_columna_nombre_de_revista/impact_factor_2014_con_nombre_revista.pdf_1729_page_2073.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2014_con_columna_nombre_de_revista/impact_factor_2014_con_nombre_revista.pdf_2074_page_2419.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2014_con_columna_nombre_de_revista/impact_factor_2014_con_nombre_revista.pdf_2420_page_2764.pdf')],
#               [
#                ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2013_con_columna_nombre_de_revista/impact_factor_2013_con_nombre_revista.pdf_1_page_340.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2013_con_columna_nombre_de_revista/impact_factor_2013_con_nombre_revista.pdf_341_page_679.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2013_con_columna_nombre_de_revista/impact_factor_2013_con_nombre_revista.pdf_680_page_1019.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2013_con_columna_nombre_de_revista/impact_factor_2013_con_nombre_revista.pdf_1020_page_1358.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2013_con_columna_nombre_de_revista/impact_factor_2013_con_nombre_revista.pdf_1359_page_1698.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2013_con_columna_nombre_de_revista/impact_factor_2013_con_nombre_revista.pdf_1699_page_2037.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2013_con_columna_nombre_de_revista/impact_factor_2013_con_nombre_revista.pdf_2038_page_2377.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2013_con_columna_nombre_de_revista/impact_factor_2013_con_nombre_revista.pdf_2378_page_2716.pdf')],
#               [
#                ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2012_con_columna_nombre_de_revista/impact_factor_2012_con_nombre_revista.pdf_1_page_336.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2012_con_columna_nombre_de_revista/impact_factor_2012_con_nombre_revista.pdf_337_page_671.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2012_con_columna_nombre_de_revista/impact_factor_2012_con_nombre_revista.pdf_672_page_1007.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2012_con_columna_nombre_de_revista/impact_factor_2012_con_nombre_revista.pdf_1008_page_1342.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2012_con_columna_nombre_de_revista/impact_factor_2012_con_nombre_revista.pdf_1343_page_1678.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2012_con_columna_nombre_de_revista/impact_factor_2012_con_nombre_revista.pdf_1679_page_2013.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2012_con_columna_nombre_de_revista/impact_factor_2012_con_nombre_revista.pdf_2014_page_2349.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2012_con_columna_nombre_de_revista/impact_factor_2012_con_nombre_revista.pdf_2350_page_2684.pdf')],
#               [
#                ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2011_con_columna_nombre_de_revista/impact_factor_2011_con_nombre_revista.pdf_1_page_251.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2011_con_columna_nombre_de_revista/impact_factor_2011_con_nombre_revista.pdf_252_page_501.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2011_con_columna_nombre_de_revista/impact_factor_2011_con_nombre_revista.pdf_502_page_879.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2011_con_columna_nombre_de_revista/impact_factor_2011_con_nombre_revista.pdf_880_page_1256.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2011_con_columna_nombre_de_revista/impact_factor_2011_con_nombre_revista.pdf_1257_page_1380.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2011_con_columna_nombre_de_revista/impact_factor_2011_con_nombre_revista.pdf_1381_page_1503.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2011_con_columna_nombre_de_revista/impact_factor_2011_con_nombre_revista.pdf_1504_page_1754.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2011_con_columna_nombre_de_revista/impact_factor_2011_con_nombre_revista.pdf_1755_page_2004.pdf')],
#               [
#                ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2010_con_columna_nombre_de_revista/impact_factor_2010_con_nombre_revista.pdf_1_page_314.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2010_con_columna_nombre_de_revista/impact_factor_2010_con_nombre_revista.pdf_315_page_628.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2010_con_columna_nombre_de_revista/impact_factor_2010_con_nombre_revista.pdf_629_page_942.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2010_con_columna_nombre_de_revista/impact_factor_2010_con_nombre_revista.pdf_943_page_1256.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2010_con_columna_nombre_de_revista/impact_factor_2010_con_nombre_revista.pdf_1257_page_1570.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2010_con_columna_nombre_de_revista/impact_factor_2010_con_nombre_revista.pdf_1571_page_1884.pdf'),
#               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2010_con_columna_nombre_de_revista/impact_factor_2010_con_nombre_revista.pdf_1885_page_2198.pdf',
#               '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2010_con_columna_nombre_de_revista/impact_factor_2010_con_nombre_revista.pdf_2199_page_2512.pdf')]
#             ]

# filename_route_out = ['/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2019_con_columna_nombre_de_revista',
#                       '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2018_con_columna_nombre_de_revista',
#                       '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2017_con_columna_nombre_de_revista',
#                       '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2016_con_columna_nombre_de_revista',
#                       '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2015_con_columna_nombre_de_revista',
#                       '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2014_con_columna_nombre_de_revista',
#                       '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2013_con_columna_nombre_de_revista',
#                       '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2012_con_columna_nombre_de_revista',
#                       '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2011_con_columna_nombre_de_revista',
#                       '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2010_con_columna_nombre_de_revista'
#                       ]

# ========================================================================
# # Años 2009-2002
# ========================================================================
year = ['2009', '2008', '2007', '2006', '2005', '2004', '2002']

files_route= [
              [
               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2009_con_columna_nombre_de_revista/impact_factor_2009_con_nombre_revista.pdf_1_page_284.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2009_con_columna_nombre_de_revista/impact_factor_2009_con_nombre_revista.pdf_285_page_567.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2009_con_columna_nombre_de_revista/impact_factor_2009_con_nombre_revista.pdf_568_page_851.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2009_con_columna_nombre_de_revista/impact_factor_2009_con_nombre_revista.pdf_852_page_1134.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2009_con_columna_nombre_de_revista/impact_factor_2009_con_nombre_revista.pdf_1135_page_1418.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2009_con_columna_nombre_de_revista/impact_factor_2009_con_nombre_revista.pdf_1419_page_1701.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2009_con_columna_nombre_de_revista/impact_factor_2009_con_nombre_revista.pdf_1702_page_1985.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2009_con_columna_nombre_de_revista/impact_factor_2009_con_nombre_revista.pdf_1986_page_2268.pdf')],
              [
               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2008_con_columna_nombre_de_revista/impact_factor_2008_con_nombre_revista.pdf_1_page_127.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2008_con_columna_nombre_de_revista/impact_factor_2008_con_nombre_revista.pdf_128_page_254.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2008_con_columna_nombre_de_revista/impact_factor_2008_con_nombre_revista.pdf_255_page_381.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2008_con_columna_nombre_de_revista/impact_factor_2008_con_nombre_revista.pdf_382_page_508.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2008_con_columna_nombre_de_revista/impact_factor_2008_con_nombre_revista.pdf_509_page_635.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2008_con_columna_nombre_de_revista/impact_factor_2008_con_nombre_revista.pdf_636_page_762.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2008_con_columna_nombre_de_revista/impact_factor_2008_con_nombre_revista.pdf_763_page_889.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2008_con_columna_nombre_de_revista/impact_factor_2008_con_nombre_revista.pdf_890_page_1016.pdf')],
              [
               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2007_con_columna_nombre_de_revista/impact_factor_2007_con_nombre_revista.pdf_1_page_160.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2007_con_columna_nombre_de_revista/impact_factor_2007_con_nombre_revista.pdf_161_page_319.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2007_con_columna_nombre_de_revista/impact_factor_2007_con_nombre_revista.pdf_320_page_479.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2007_con_columna_nombre_de_revista/impact_factor_2007_con_nombre_revista.pdf_480_page_638.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2007_con_columna_nombre_de_revista/impact_factor_2007_con_nombre_revista.pdf_639_page_798.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2007_con_columna_nombre_de_revista/impact_factor_2007_con_nombre_revista.pdf_799_page_957.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2007_con_columna_nombre_de_revista/impact_factor_2007_con_nombre_revista.pdf_958_page_1117.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2007_con_columna_nombre_de_revista/impact_factor_2007_con_nombre_revista.pdf_1118_page_1276.pdf')],
              [
               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2006_con_columna_nombre_de_revista/impact_factor_2006_con_nombre_revista.pdf_1_page_119.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2006_con_columna_nombre_de_revista/impact_factor_2006_con_nombre_revista.pdf_120_page_238.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2006_con_columna_nombre_de_revista/impact_factor_2006_con_nombre_revista.pdf_239_page_357.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2006_con_columna_nombre_de_revista/impact_factor_2006_con_nombre_revista.pdf_358_page_476.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2006_con_columna_nombre_de_revista/impact_factor_2006_con_nombre_revista.pdf_477_page_595.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2006_con_columna_nombre_de_revista/impact_factor_2006_con_nombre_revista.pdf_596_page_714.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2006_con_columna_nombre_de_revista/impact_factor_2006_con_nombre_revista.pdf_715_page_833.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2006_con_columna_nombre_de_revista/impact_factor_2006_con_nombre_revista.pdf_834_page_952.pdf')],
              [
               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2005_con_columna_nombre_de_revista/impact_factor_2005_con_nombre_revista.pdf_1_page_118.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2005_con_columna_nombre_de_revista/impact_factor_2005_con_nombre_revista.pdf_119_page_235.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2005_con_columna_nombre_de_revista/impact_factor_2005_con_nombre_revista.pdf_236_page_353.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2005_con_columna_nombre_de_revista/impact_factor_2005_con_nombre_revista.pdf_354_page_470.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2005_con_columna_nombre_de_revista/impact_factor_2005_con_nombre_revista.pdf_471_page_588.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2005_con_columna_nombre_de_revista/impact_factor_2005_con_nombre_revista.pdf_589_page_705.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2005_con_columna_nombre_de_revista/impact_factor_2005_con_nombre_revista.pdf_706_page_823.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2005_con_columna_nombre_de_revista/impact_factor_2005_con_nombre_revista.pdf_824_page_940.pdf')],
              [
               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2004_con_columna_nombre_de_revista/impact_factor_2004_con_nombre_revista.pdf_1_page_115.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2004_con_columna_nombre_de_revista/impact_factor_2004_con_nombre_revista.pdf_116_page_230.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2004_con_columna_nombre_de_revista/impact_factor_2004_con_nombre_revista.pdf_231_page_345.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2004_con_columna_nombre_de_revista/impact_factor_2004_con_nombre_revista.pdf_346_page_460.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2004_con_columna_nombre_de_revista/impact_factor_2004_con_nombre_revista.pdf_461_page_575.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2004_con_columna_nombre_de_revista/impact_factor_2004_con_nombre_revista.pdf_576_page_690.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2004_con_columna_nombre_de_revista/impact_factor_2004_con_nombre_revista.pdf_691_page_805.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2004_con_columna_nombre_de_revista/impact_factor_2004_con_nombre_revista.pdf_806_page_920.pdf')],
              [
               ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2002_con_columna_nombre_de_revista/impact_factor_2002_con_nombre_revista.pdf_1_page_190.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2002_con_columna_nombre_de_revista/impact_factor_2002_con_nombre_revista.pdf_191_page_380.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2002_con_columna_nombre_de_revista/impact_factor_2002_con_nombre_revista.pdf_381_page_570.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2002_con_columna_nombre_de_revista/impact_factor_2002_con_nombre_revista.pdf_571_page_760.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2002_con_columna_nombre_de_revista/impact_factor_2002_con_nombre_revista.pdf_761_page_950.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2002_con_columna_nombre_de_revista/impact_factor_2002_con_nombre_revista.pdf_951_page_1140.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2002_con_columna_nombre_de_revista/impact_factor_2002_con_nombre_revista.pdf_1141_page_1330.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2002_con_columna_nombre_de_revista/impact_factor_2002_con_nombre_revista.pdf_1331_page_1520.pdf')],
              
            ]

filename_route_out = ['/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2009_con_columna_nombre_de_revista',
                      '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2008_con_columna_nombre_de_revista',
                      '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2007_con_columna_nombre_de_revista',
                      '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2006_con_columna_nombre_de_revista',
                      '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2005_con_columna_nombre_de_revista',
                      '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2004_con_columna_nombre_de_revista',
                      '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2002_con_columna_nombre_de_revista'
                      ]

def generating_final_csv(files_route, filename_route_out, year_):

  for i in np.arange(0,len(files_route)):
    ### ==============================
    # Ajustando el rango del FOR se puede ajustar el nº de CSV que se genera. 
    # Esto puede ser útil si sólo se quiere generar el CSV nº 3, o el 0, por ejemplo,
    # con for i in np.arange(1,4): estaríamos generando los csv 1-3, dejando el 0 fuera.
    ### ==============================

    print('\nReading the following pdf file: {}'.format(files_route[i][0]))
    table = load_pdf_into_tables(files_route[i][0])
    print('PDF 1 converted into table!')

    df_1 = df_joined_rows(table)
    print('Tables converted into DataFrame 1!')
    table = None # esto es importante para que la RAM no aumente hasta el límite
    # y bloquee la ejecución.

    print('Reading the following pdf file: {}'.format(files_route[i][1]))
    table = load_pdf_into_tables(files_route[i][1])
    print('PDF 1 converted into table!')

    df_2 = df_joined_rows(table)
    print('Tables converted into DataFrame 2!')

    print('Proceeding to concatenate df_1 and df_2')
    df_final = concatenated_df(df_1,df_2)
    table = None

    save_df_to_csv(df_final,filename_route_out, year = year_, i=i )

## NOTA 1: habilitar la OPCIÓN 1 si el PDF no contiene el nombre de la revista
## NOTA 2: habilitar la OPCIÓN 2 si el PDF si contiene el nombre de la revista
    # if i == 0:
      ##############
      ## OPCIÓN 1 ##
      ##############
      ## Esta opción arregla la columna CATEGORY_DESCRIPTION, aunque posteriormente se determinó que no hacía falta, 
      ## luego, no aparecerá en los PDFs y por tanto no haría falta arreglar esta columna


    #   df_final = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020_nueva_columna/Impact_factor_2020_0.csv')
    # #  print('Fixing CATEGORY_DESCRIPTION column!')
    # #  categories = df_final['CATEGORY_DESCRIPTION'].unique() # también guarda los nan
    # #  categories = list(filter(lambda x: pd.isna(x) is not True, categories)) # eliminamos los nan de categories
    # #  df_final = replace_nan_values_in_category_description(df_final, categories)
    # #  print('Category description column fixed!')

    if i == 0:
      ##############
      ## OPCIÓN 2 ##
      ##############

      # Sustituir la ruta indicada por la correcta para cada caso, esto es, tener en 
      # cuenta que la ruta indicada puede ser diferente en cada Drive.
      
      df_final = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_' + year_ + '_con_columna_nombre_de_revista/Impact_factor_' + year_ + '_0.csv')
      
    elif i == 1:
      df_final = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_' + year_ + '_con_columna_nombre_de_revista/Impact_factor_' + year_ + '_1.csv')
    
    elif i == 2:
      df_final = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_' + year_ + '_con_columna_nombre_de_revista/Impact_factor_' + year_ + '_2.csv')

    else:
      '''
      Si queremos transformar el pdf con el nombre de la revista, habilitamos este elif, sino lo comentamos, ya que sólo se generarían 3 csv (0,1 y 2)
      '''
      df_final = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_' + year_ + '_con_columna_nombre_de_revista/Impact_factor_' + year_ + '_3.csv')


    print('Fixing the index and title columns.')
    # Ver la función extract_index_text() anteriormente explicada
    df_final = extract_index_text(999, df_final)
    
    print('Proceeding to save the final df')
    print('Size of df: {}\n'. format(df_final.shape))

    
    save_df_to_csv(df_final,filename_route_out, year = year_, i=i )


In [ ]:
#############################################################
# Bucle que recorre las variables anteriormente declaradas y va extrayendo
# los CSV a partir de cada PDF previamente troceado.
#############################################################
for k in range(len(files_route)):
  print(k)
  generating_final_csv(files_route[k], filename_route_out[k], year[k])

0

Reading the following pdf file: /content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2009_con_columna_nombre_de_revista/impact_factor_2009_con_nombre_revista.pdf_1_page_284.pdf
Total tables extracted: 284
PDF 1 converted into table!
Tables converted into DataFrame 1!
Reading the following pdf file: /content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2009_con_columna_nombre_de_revista/impact_factor_2009_con_nombre_revista.pdf_285_page_567.pdf
Total tables extracted: 283
PDF 1 converted into table!
Tables converted into DataFrame 2!
Proceeding to concatenate df_1 and df_2
Dataframe concatenated!
CSV saved!
Fixing the index and title columns.
Proceeding to save the final df
Size of df: (14726, 5)

CSV saved!

Reading the following pdf file: /content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2009_con_columna_nombre_de_revista/impact_factor_2009_con_nombre_revista.pdf_568_page_851.pdf
Total tables extracted: 284
P

**A continuación está el código adaptado para un solo caso**

In [ ]:
###############################################
# Aquí se tendría que definir el año, las rutas a los PDFs troceados y la ruta a 
# la carpeta de salida
###############################################

year = ['2020']

files_route= [('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020_nueva_columna/Impact_factor_2020_columna_titulo.pdf_1_page_385.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020_nueva_columna/Impact_factor_2020_columna_titulo.pdf_386_page_770.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020_nueva_columna/Impact_factor_2020_columna_titulo.pdf_771_page_1155.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020_nueva_columna/Impact_factor_2020_columna_titulo.pdf_1156_page_1540.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020_nueva_columna/Impact_factor_2020_columna_titulo.pdf_1541_page_1925.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020_nueva_columna/Impact_factor_2020_columna_titulo.pdf_1926_page_2310.pdf'),
              ('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020_nueva_columna/Impact_factor_2020_columna_titulo.pdf_2311_page_2695.pdf',
              '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020_nueva_columna/Impact_factor_2020_columna_titulo.pdf_2696_page_3080.pdf')]

filename_route_out = '/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020_nueva_columna'

def generating_final_csv(files_route, filename_route_out, year_):

  ### ===============
  # Ajustar los rangos para hacer 1, 2 o 3 .csv
  ### ===============

  for i in np.arange(0,len(files_route)):
  # for i in np.arange(2,3):
    print('\nReading the following pdf file: {}'.format(files_route[i][0]))
    table = load_pdf_into_tables(files_route[i][0])
    print('PDF 1 converted into table!')

    df_1 = df_joined_rows(table)
    print('Tables converted into DataFrame 1!')
    table = None

    print('Reading the following pdf file: {}'.format(files_route[i][1]))
    table = load_pdf_into_tables(files_route[i][1])
    print('PDF 1 converted into table!')

    df_2 = df_joined_rows(table)
    print('Tables converted into DataFrame 2!')

    print('Proceeding to concatenate df_1 and df_2')
    df_final = concatenated_df(df_1,df_2)
    table = None

    save_df_to_csv(df_final,filename_route_out, year = year_, i=i )

## NOTA: ojo a dejar el if de forma correcta, es decir, primero el 0 y luego el resto
## NOTA 2: habilitar la OPCIÓN 1 si el PDF no contiene el nombre de la revista
## NOTA 3: habilitar la OPCIÓN 2 si el PDF si contiene el nombre de la revista
    # if i == 0:
      ##############
      ## OPCIÓN 1 ##
      ##############
      ## Esta opción arregla la columna CATEGORY_DESCRIPTION, aunque posteriormente se determinó que no hacía falta, 
      ## luego, no aparecerá en los PDFs y por tanto no haría falta arreglar esta columna

    #   df_final = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020_nueva_columna/Impact_factor_2020_0.csv')
    # #  print('Fixing CATEGORY_DESCRIPTION column!')
    # #  categories = df_final['CATEGORY_DESCRIPTION'].unique() # también guarda los nan
    # #  categories = list(filter(lambda x: pd.isna(x) is not True, categories)) # eliminamos los nan de categories
    # #  df_final = replace_nan_values_in_category_description(df_final, categories)
    # #  print('Category description column fixed!')

    if i == 0:
      ##############
      ## OPCIÓN 2 ##
      ##############

      # Sustituir la ruta indicada por la correcta para cada caso, esto es, tener en 
      # cuenta que la ruta indicada puede ser diferente en cada Drive.

      df_final = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_' + year_ +'_nueva_columna/Impact_factor_' + year_ +'_0.csv')
      
    elif i == 1:
      df_final = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_' + year_ +'_nueva_columna/Impact_factor_' + year_ +'_1.csv')
    
    elif i == 2:
      df_final = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_' + year_ +'_nueva_columna/Impact_factor_' + year_ +'_2.csv')

    else:
      '''
      Si hay 4 cortes de columna, habilitamos este else, sino lo comentamos, ya que sólo se generarían 3 csv (0,1 y 2)
      '''
      df_final = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_' + year_ +'_nueva_columna/Impact_factor_' + year_ +'_3.csv')


    print('Fixing the index and title columns.')
    df_final = extract_index_text(999, df_final)
    
    print('Proceeding to save the final df')
    print('Size of df: {}\n'. format(df_final.shape))

    
    save_df_to_csv(df_final,filename_route_out, year = year_, i=i )




In [ ]:
generating_final_csv(files_route, filename_route_out, '2020')


Reading the following pdf file: /content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020_nueva_columna/Impact_factor_2020_columna_titulo.pdf_1_page_385.pdf
Total tables extracted: 385
PDF 1 converted into table!
Tables converted into DataFrame 1!
Reading the following pdf file: /content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020_nueva_columna/Impact_factor_2020_columna_titulo.pdf_386_page_770.pdf
Total tables extracted: 385
PDF 1 converted into table!
Tables converted into DataFrame 2!
Proceeding to concatenate df_1 and df_2
Dataframe concatenated!
CSV saved!
Fixing the index and title columns.
Proceeding to save the final df
Size of df: (20005, 5)

CSV saved!

Reading the following pdf file: /content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020_nueva_columna/Impact_factor_2020_columna_titulo.pdf_771_page_1155.pdf
Total tables extracted: 385
PDF 1 converted into table!
Tables converted into DataFrame 1!

# Leemos todos los csv generados, estudiamos los nulos de cada uno y creamos el csv final

In [ ]:
df_1 = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020_nueva_columna/Impact_factor_2020_0.csv')
df_2 = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020_nueva_columna/Impact_factor_2020_1.csv')
df_3 = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020_nueva_columna/Impact_factor_2020_2.csv')
df_4 = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020_nueva_columna/Impact_factor_2020_3.csv')

df_1.head(10)
print(df_1.isnull().sum()/df_1.shape[0])
print(df_2.isnull().sum()/df_2.shape[0])
print(df_3.isnull().sum()/df_3.shape[0])
print(df_4.isnull().sum()/df_4.shape[0])

df_final = create_final_csv(df_1, df_2, df_3,df_4)
df_final.head()


Unnamed: 0              0.0
Title20                 0.0
Year                    0.0
ISSN                    0.0
CATEGORY_DESCRIPTION    0.0
dtype: float64
Unnamed: 0                        0.000000
Title20                           0.000000
Publisher                         0.060885
TOT_CITES                         0.174106
IMPACT_FACTOR                     0.040990
Number of Journals in Category    0.009198
dtype: float64
Unnamed: 0           0.0
Title20              0.0
ARTL_INFLUENCE       0.0
CATEGORY_RANKING     0.0
IMMEDIACY_INDEX      0.0
CITED_HALF_LIFE      0.0
5YR_IMPACT_FACTOR    0.0
dtype: float64
Unnamed: 0               0.0
Title20                  0.0
EIGENFACTOR              0.0
Rank by Impact Factor    0.0
QUARTILE_RANK            0.0
dtype: float64


,Unnamed: 0,Title20,Year,ISSN,CATEGORY_DESCRIPTION,Publisher,TOT_CITES,IMPACT_FACTOR,Number of Journals in Category,ARTL_INFLUENCE,CATEGORY_RANKING,IMMEDIACY_INDEX,CITED_HALF_LIFE,5YR_IMPACT_FACTOR,EIGENFACTOR,Rank by Impact Factor,QUARTILE_RANK
0,1,2D MATER,2020,2053-1583,"MATERIALS SCIENCE, MULTIDISCIPLINARY",2D Materials,9244,"7,103",N/D,"2,167",Q1,"2,404","3,5","8,161","0,02239",N/D,72/334
1,2,3 BIOTECH,2020,2190-572X,BIOTECHNOLOGY & APPLIED MICROBIOLOGY,3 Biotech,6625,"2,406",N/D,"0,469",Q3,"0,542","3,6","3,203","0,00822",N/D,113/159
2,3,3D PRINT ADDIT MANUF,2020,2329-7662,"MATERIALS SCIENCE, MULTIDISCIPLINARY",3D Printing and Additive Manufacturing,1123,"5,449",N/D,"1,068",Q2,"1,051",4,"5,48","0,00167",N/D,96/334
3,4,3D PRINT ADDIT MANUF,2020,2329-7662,"ENGINEERING, MANUFACTURING",3D Printing and Additive Manufacturing,1123,"5,449",N/D,"1,068",Q2,"1,051",4,"5,48","0,00167",N/D,14/50
4,5,4OR-Q J OPER RES,2020,1619-4500,OPERATIONS RESEARCH & MANAGEMENT SCIENCE,4OR-A Quarterly Journal of Operations Research,678,"1,545",N/D,"0,784",Q3,"0,477","9,6","2,176","0,00068",N/D,63/84


In [ ]:
################################################
# Limpiamos de nulo y agregamos los vlaores correctos a la columna
# del nombre de la revista, ya que es sólo la que nos interesa.
################################################
df_publisher_column_fixed = fixing_publisher_column(df_final)

##### paso 1 #####
Nan element (PUBLISHER):  nan
Updated element:  ABHANDLUNGEN AUS DEM MATHEMATISCHEN SEMINAR DER UNIVERSITAT HAMBURG
Nan element (PUBLISHER):  nan
Updated element:  ACM Transactions on Asian and Low-Resource Language Information Processing
Nan element (PUBLISHER):  nan
Updated element:  ACM TRANSACTIONS ON DESIGN AUTOMATION OF ELECTRONIC SYSTEMS
Nan element (PUBLISHER):  nan
Updated element:  ACM TRANSACTIONS ON DESIGN AUTOMATION OF ELECTRONIC SYSTEMS
Nan element (PUBLISHER):  nan
Updated element:  ACM Transactions on Multimedia Computing Communications and Applications
Nan element (PUBLISHER):  nan
Updated element:  ACM Transactions on Multimedia Computing Communications and Applications
Nan element (PUBLISHER):  nan
Updated element:  ACM Transactions on Multimedia Computing Communications and Applications
Nan element (PUBLISHER):  nan
Updated element:  ACM TRANSACTIONS ON PROGRAMMING LANGUAGES AND SYSTEMS
Nan element (PUBLISHER):  nan
Updated element:  ACM TRANSACTI

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Se han truncado las últimas 5000 líneas del flujo de salida.
Nan element (PUBLISHER):  nan
Updated element:  CANADIAN JOURNAL OF AGRICULTURAL ECONOMICS-REVUE CANADIENNE D AGROECONOMIE
Nan element (PUBLISHER):  nan
Updated element:  CANADIAN JOURNAL OF BEHAVIOURAL SCIENCE-REVUE CANADIENNE DES SCIENCES DU COMPORTEMENT
Nan element (PUBLISHER):  nan
Updated element:  CANADIAN JOURNAL OF CRIMINOLOGY AND CRIMINAL JUSTICE
Nan element (PUBLISHER):  nan
Updated element:  CANADIAN JOURNAL OF DEVELOPMENT STUDIES-REVUE CANADIENNE D ETUDES DU DEVELOPPEMENT
Nan element (PUBLISHER):  nan
Updated element:  CANADIAN JOURNAL OF ECONOMICS-REVUE CANADIENNE D ECONOMIQUE
Nan element (PUBLISHER):  nan
Updated element:  CANADIAN JOURNAL OF ELECTRICAL AND COMPUTER ENGINEERING-REVUE CANADIENNE DE GENIE ELECTRIQUE ...
Nan element (PUBLISHER):  nan
Updated element:  CANADIAN JOURNAL OF ELECTRICAL AND COMPUTER ENGINEERING-REVUE CANADIENNE DE GENIE ELECTRIQUE ...
Nan element (PUBLISHER):  nan
Updated element:  CANA

In [ ]:
df_publisher_column_fixed.iloc[:,[1,2,3,5]].to_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020_nueva_columna/IF_nombres_revistas_2020.csv', 
                                                   index = False, sep = ';')

In [ ]:
# df_publisher_column_fixed[df_publisher_column_fixed['Publisher'].isna()]
# df_publisher_column_fixed.iloc[18330,5] = 'ZYGOTE'
# df_publisher_column_fixed.isna().sum()/len(df_publisher_column_fixed)


# df_publisher_column_fixed.iloc[10706,3] = 'N/D'  = 'No está en PDF'
df_publisher_column_fixed[df_publisher_column_fixed['ISSN'].isna()]

fix_index = df_publisher_column_fixed[df_publisher_column_fixed['ISSN'].isna()].index
for i in fix_index:
  df_publisher_column_fixed.iloc[i,3] = 'Not in PDF'

df_publisher_column_fixed[df_publisher_column_fixed['ISSN'] == 'Not in PDF']


,Unnamed: 0,Title20,Year,ISSN,CATEGORY_DESCRIPTION,Publisher,TOT_CITES,IMPACT_FACTOR,Number of Journals in Category,ARTL_INFLUENCE,CATEGORY_RANKING,IMMEDIACY_INDEX,CITED_HALF_LIFE,5YR_IMPACT_FACTOR,EIGENFACTOR,Rank by Impact Factor,QUARTILE_RANK
3935,3936,INDIAN J CHEM A,2002,Not in PDF,N/D,INDIAN JOURNAL OF CHEMISTRY SECTION A-INORGANI...,INDIAN JOURNAL OF CHEMISTRY SECTION A-INORGANI...,NaN,NaN,N/D,N/D,"0,024","9,3",N/D,N/D,N/D,N/D


In [ ]:
 ### CUANDO tot_cites es nan, las citas están incrustadas en publisher 
### cuando publisher es nan, la revista está inscrustada en tot_cites
### no coinciden los nan en tot_cites e impact_factor

# df_final[(df_final['IMPACT_FACTOR'].isnull()) & (df_final['Publisher'].isnull()) ]
df_final[(df_final['Number of Journals in Category'].isnull()) ]

,Unnamed: 0,Title20,Year,ISSN,CATEGORY_DESCRIPTION,Publisher,TOT_CITES,IMPACT_FACTOR,Number of Journals in Category,ARTL_INFLUENCE,CATEGORY_RANKING,IMMEDIACY_INDEX,CITED_HALF_LIFE,5YR_IMPACT_FACTOR,EIGENFACTOR,Rank by Impact Factor,QUARTILE_RANK
707,708,AI EDAM,2020,0890-0604,"ENGINEERING, MULTIDISCIPLINARY",NaN,AI EDAM-ARTIFICIAL INTELLIGENCE FOR ENGINEERIN...,NaN,NaN,"0,352",Q3,"0,371","9,1","1,7","0,00061",N/D,58/90
708,709,AI EDAM,2020,0890-0604,"COMPUTER SCIENCE, INTERDISCIPLINARY APPLICATIONS",NaN,AI EDAM-ARTIFICIAL INTELLIGENCE FOR ENGINEERIN...,NaN,NaN,"0,352",Q4,"0,371","9,1","1,7","0,00061",N/D,90/111
709,710,AI EDAM,2020,0890-0604,"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",NaN,AI EDAM-ARTIFICIAL INTELLIGENCE FOR ENGINEERIN...,NaN,NaN,"0,352",Q4,"0,371","9,1","1,7","0,00061",N/D,105/139
710,711,AI EDAM,2020,0890-0604,"ENGINEERING, MANUFACTURING",NaN,AI EDAM-ARTIFICIAL INTELLIGENCE FOR ENGINEERIN...,NaN,NaN,"0,352",Q4,"0,371","9,1","1,7","0,00061",N/D,44/50
910,911,AM J MED GENET C,2020,1552-4868,GENETICS & HEREDITY,NaN,AMERICAN JOURNAL OF MEDICAL GENETICS PART C-SE...,NaN,NaN,"1,87",Q2,"1,098","6,4","6,018","0,00423",N/D,72/176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19937,19938,Z NATURFORSCH A,2020,0932-0784,"CHEMISTRY, PHYSICAL",NaN,ZEITSCHRIFT FUR NATURFORSCHUNG SECTION A-A JOU...,NaN,NaN,"0,213",Q4,"0,49","14,3","1,192","0,00138",N/D,144/162
19938,19939,Z NATURFORSCH A,2020,0932-0784,"PHYSICS, MULTIDISCIPLINARY",NaN,ZEITSCHRIFT FUR NATURFORSCHUNG SECTION A-A JOU...,NaN,NaN,"0,213",Q3,"0,49","14,3","1,192","0,00138",N/D,59/86
19939,19940,Z NATURFORSCH B,2020,0932-0776,"CHEMISTRY, ORGANIC",NaN,ZEITSCHRIFT FUR NATURFORSCHUNG SECTION B-A JOU...,NaN,NaN,"0,129",Q4,"0,397","17,2","0,945","0,00084",N/D,48/57
19940,19941,Z NATURFORSCH B,2020,0932-0776,"CHEMISTRY, INORGANIC & NUCLEAR",NaN,ZEITSCHRIFT FUR NATURFORSCHUNG SECTION B-A JOU...,NaN,NaN,"0,129",Q4,"0,397","17,2","0,945","0,00084",N/D,42/45


##### paso 1 #####
Nan element (PUBLISHER):  nan
Updated element:  ABHANDLUNGEN AUS DEM MATHEMATISCHEN SEMINAR DER UNIVERSITAT HAMBURG
Nan element (PUBLISHER):  nan
Updated element:  ACM Transactions on Asian and Low-Resource Language Information Processing
Nan element (PUBLISHER):  nan
Updated element:  ACM TRANSACTIONS ON DESIGN AUTOMATION OF ELECTRONIC SYSTEMS
Nan element (PUBLISHER):  nan
Updated element:  ACM TRANSACTIONS ON DESIGN AUTOMATION OF ELECTRONIC SYSTEMS
Nan element (PUBLISHER):  nan
Updated element:  ACM Transactions on Multimedia Computing Communications and Applications
Nan element (PUBLISHER):  nan
Updated element:  ACM Transactions on Multimedia Computing Communications and Applications
Nan element (PUBLISHER):  nan
Updated element:  ACM Transactions on Multimedia Computing Communications and Applications
Nan element (PUBLISHER):  nan
Updated element:  ACM TRANSACTIONS ON PROGRAMMING LANGUAGES AND SYSTEMS
Nan element (PUBLISHER):  nan
Updated element:  ACM TRANSACTI

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Se han truncado las últimas 5000 líneas del flujo de salida.
Nan element (PUBLISHER):  nan
Updated element:  CANADIAN JOURNAL OF ELECTRICAL AND COMPUTER ENGINEERING-REVUE CANADIENNE DE GENIE ELECTRIQUE ...
Nan element (PUBLISHER):  nan
Updated element:  CANADIAN JOURNAL OF EXPERIMENTAL PSYCHOLOGY-REVUE CANADIENNE DE PSYCHOLOGIE EXPERIMENTALE
Nan element (PUBLISHER):  nan
Updated element:  CANADIAN JOURNAL OF INFORMATION AND LIBRARY SCIENCE-REVUE CANADIENNE DES SCIENCES DE L INFORM...
Nan element (PUBLISHER):  nan
Updated element:  CANADIAN JOURNAL OF MATHEMATICS-JOURNAL CANADIEN DE MATHEMATIQUES
Nan element (PUBLISHER):  nan
Updated element:  Canadian Journal of Occupational Therapy-Revue Canadienne d Ergotherapie
Nan element (PUBLISHER):  nan
Updated element:  Canadian Journal of Occupational Therapy-Revue Canadienne d Ergotherapie
Nan element (PUBLISHER):  nan
Updated element:  CANADIAN JOURNAL OF OPHTHALMOLOGY-JOURNAL CANADIEN D OPHTALMOLOGIE
Nan element (PUBLISHER):  nan
Updated el

In [ ]:
df_publisher_column_fixed.iloc[:, 1:6].to_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020/impact_factor_2020_con_nombre_revista.csv',
                                              index = False, sep = ';')

In [ ]:
type(df_publisher_column_fixed[df_publisher_column_fixed['Publisher'].isnull()]['IMPACT_FACTOR'].iloc[1])

str

In [ ]:
df_final_copy = df_final.copy()
for j in np.arange(0,len(df_final_copy)):
  if type(df_final_copy['Publisher'].iloc[j]) == str:
    # print('Str element: ', df_final['Publisher'].iloc[j])
    pass
  elif (type(df_final_copy['Publisher'].iloc[j]) == float) & (type(df_final_copy['TOT_CITES'].iloc[j]) == str) & (type(df_final_copy['IMPACT_FACTOR'].iloc[j]) == float):
    print('\n### Primera transformación ###', j)
    print('Nan element (PUBLISHER): ', df_final_copy['Publisher'].iloc[j])
    df_final_copy['Publisher'].iloc[j] = df_final_copy['TOT_CITES'].iloc[j]
    print('Updated element: ', df_final_copy['Publisher'].iloc[j])


  
  # elif (type(df_final_copy['Publisher'].iloc[j]) == float) & (type(df_final_copy['IMPACT_FACTOR'].iloc[j]) == float):
  #   print('### 2 Segunda transformación ###', j)
  #   print('Nan element (IMPACT_FACTOR): ', df_final_copy['IMPACT_FACTOR'].iloc[j])
  #   print('Nan element (PUBLISHER): ', df_final_copy['Publisher'].iloc[j])
  #   df_final_copy['Publisher'].iloc[j] = df_final_copy['TOT_CITES'].iloc[j]
  #   print('Updated element: ', df_final_copy['Publisher'].iloc[j])


### Primera transformación ### 14
Nan element (PUBLISHER):  nan
Updated element:  ABHANDLUNGEN AUS DEM MATHEMATISCHEN SEMINAR DER UNIVERSITAT HAMBURG
### Primera transformación ### 66
Nan element (PUBLISHER):  nan
Updated element:  ACM Transactions on Asian and Low-Resource Language Information Processing
### Primera transformación ### 78
Nan element (PUBLISHER):  nan
Updated element:  ACM TRANSACTIONS ON DESIGN AUTOMATION OF ELECTRONIC SYSTEMS
### Primera transformación ### 79
Nan element (PUBLISHER):  nan
Updated element:  ACM TRANSACTIONS ON DESIGN AUTOMATION OF ELECTRONIC SYSTEMS
### Primera transformación ### 95
Nan element (PUBLISHER):  nan
Updated element:  ACM Transactions on Multimedia Computing Communications and Applications
### Primera transformación ### 96
Nan element (PUBLISHER):  nan
Updated element:  ACM Transactions on Multimedia Computing Communications and Applications
### Primera transformación ### 97
Nan element (PUBLISHER):  nan
Updated element:  ACM Transactions 

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)



### Primera transformación ### 2775
Nan element (PUBLISHER):  nan
Updated element:  BOUNDARY 2-AN INTERNATIONAL JOURNAL OF LITERATURE AND CULTURE
### Primera transformación ### 2890
Nan element (PUBLISHER):  nan
Updated element:  BRITISH JOURNAL OF MATHEMATICAL & STATISTICAL PSYCHOLOGY
### Primera transformación ### 2891
Nan element (PUBLISHER):  nan
Updated element:  BRITISH JOURNAL OF MATHEMATICAL & STATISTICAL PSYCHOLOGY
### Primera transformación ### 2892
Nan element (PUBLISHER):  nan
Updated element:  BRITISH JOURNAL OF MATHEMATICAL & STATISTICAL PSYCHOLOGY
### Primera transformación ### 2893
Nan element (PUBLISHER):  nan
Updated element:  BRITISH JOURNAL OF MATHEMATICAL & STATISTICAL PSYCHOLOGY
### Primera transformación ### 2974
Nan element (PUBLISHER):  nan
Updated element:  CALIFORNIA COOPERATIVE OCEANIC FISHERIES INVESTIGATIONS REPORTS
### Primera transformación ### 2975
Nan element (PUBLISHER):  nan
Updated element:  CALCULUS OF VARIATIONS AND PARTIAL DIFFERENTIAL EQUATIONS

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20005 entries, 0 to 20004
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Unnamed: 0                      20005 non-null  object
 1   Title20                         20005 non-null  object
 2   Year                            20005 non-null  int64 
 3   ISSN                            20005 non-null  object
 4   CATEGORY_DESCRIPTION            20005 non-null  object
 5   Publisher                       18787 non-null  object
 6   TOT_CITES                       16522 non-null  object
 7   IMPACT_FACTOR                   19185 non-null  object
 8   Number of Journals in Category  19821 non-null  object
 9   ARTL_INFLUENCE                  20005 non-null  object
 10  CATEGORY_RANKING                20005 non-null  object
 11  IMMEDIACY_INDEX                 20005 non-null  object
 12  CITED_HALF_LIFE                 20005 non-null

In [ ]:
np.arange(0,len(df_final))

array([    0,     1,     2, ..., 20002, 20003, 20004])

In [ ]:
type(df_final['IMPACT_FACTOR'].iloc[19954]) == float

True

In [ ]:
df_final['IMPACT_FACTOR'].iloc[0]

'7,103'

In [ ]:
df_final.shape

(20005, 17)

In [ ]:
save_df_to_csv(df_final, filename_route_out='/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2004',
               year = '2004', i='final')

CSV saved!


In [ ]:
df_test = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2008/Impact_factor_2008_final.csv')
# df_test = df_final.copy()
# df_test = df_test[df_test['ISSN'].isnull()].iloc[:, 3] = 'N/D'
df_test['ISSN'].fillna('N/D', inplace = True)
print(df_test.isnull().sum()/df_test.shape[0])
df_test[df_test['ISSN'] == 'N/D']
# df_test[df_test['ISSN'].isnull()].iloc[:, 3] 

Unnamed: 0                        0.0
Title20                           0.0
Year                              0.0
ISSN                              0.0
Categories                        0.0
CATEGORY_DESCRIPTION              0.0
TOT_CITES                         0.0
IMPACT_FACTOR                     0.0
Number of Journals in Category    0.0
ARTL_INFLUENCE                    0.0
CATEGORY_RANKING                  0.0
IMMEDIACY_INDEX                   0.0
CITED_HALF_LIFE                   0.0
5YR_IMPACT_FACTOR                 0.0
EIGENFACTOR                       0.0
Rank by Impact Factor             0.0
QUARTILE_RANK                     0.0
dtype: float64


,Unnamed: 0,Title20,Year,ISSN,Categories,CATEGORY_DESCRIPTION,TOT_CITES,IMPACT_FACTOR,Number of Journals in Category,ARTL_INFLUENCE,CATEGORY_RANKING,IMMEDIACY_INDEX,CITED_HALF_LIFE,5YR_IMPACT_FACTOR,EIGENFACTOR,Rank by Impact Factor,QUARTILE_RANK
1775,1776,COMPEL,2008,N/D,N/D,N/D,333,"0,441",N/D,N/D,N/D,"0,138",4,N/D,N/D,N/D,N/D
3409,3410,INDIAN J CHEM A,2008,N/D,N/D,N/D,2046,"0,575",N/D,N/D,N/D,"0,117","99,9",N/D,N/D,N/D,N/D
4460,4461,J ENVIRON SCI HEAL A,2008,N/D,N/D,N/D,1885,"1,002",N/D,N/D,N/D,"0,164","5,4",N/D,N/D,N/D,N/D
5231,5232,J S AFR VET ASSOC,2008,N/D,N/D,N/D,540,"0,398",N/D,N/D,N/D,0,"99,9",N/D,N/D,N/D,N/D
6475,6476,P I MECH ENG L-J MAT,2008,N/D,N/D,N/D,97,"0,451",N/D,N/D,N/D,0,N/D,N/D,N/D,N/D,N/D
6860,6861,PRECIS ENG,2008,N/D,N/D,N/D,1019,"0,895",N/D,N/D,N/D,"0,395","8,6",N/D,N/D,N/D,N/D


In [ ]:
save_df_to_csv(df_test, filename_route_out='/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2008',
               year = '2008', i='final')

CSV saved!


In [ ]:
df_1[df_1['ISSN'].isnull()].iloc[:, 3] 

1775    NaN
3409    NaN
4460    NaN
5231    NaN
6475    NaN
6860    NaN
Name: ISSN, dtype: object

In [ ]:
df_test_final = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2008/Impact_factor_2008_final.csv')


df_test_final.head(10)
print(df_test_final.isnull().sum()/df_test_final.shape[0])


Unnamed: 0                        0.000000
Title20                           0.000000
Year                              0.000000
ISSN                              0.000731
Categories                        0.000000
CATEGORY_DESCRIPTION              0.000000
TOT_CITES                         0.000000
IMPACT_FACTOR                     0.000000
Number of Journals in Category    0.000000
ARTL_INFLUENCE                    0.000000
CATEGORY_RANKING                  0.000000
IMMEDIACY_INDEX                   0.000000
CITED_HALF_LIFE                   0.000000
5YR_IMPACT_FACTOR                 0.000000
EIGENFACTOR                       0.000000
Rank by Impact Factor             0.000000
QUARTILE_RANK                     0.000000
dtype: float64


In [ ]:
categories = df_2['CATEGORY_DESCRIPTION'].unique() # también guarda los nan
categories = list(filter(lambda x: pd.isna(x) is not True, categories)) # eliminamos los nan de categories
df_final = replace_nan_values_in_category_description(df_2, categories)
df_final.isnull().sum()/df_final.shape[0]

Unnamed: 0              0.000000
Title20                 0.000000
Year                    0.000000
ISSN                    0.000000
Categories              0.003105
CATEGORY_DESCRIPTION    0.000000
TOT_CITES               0.000000
dtype: float64

In [ ]:
df_final[df_final['Categories'].isnull()]

,Unnamed: 0,Title20,Year,ISSN,Categories,CATEGORY_DESCRIPTION,TOT_CITES
397,398,ADV FUNCT MATER,2018,1616-301X,NaN,"MATERIALS SCIENCE, MULTIDISCIPLINARY",95431
398,399,ADV FUNCT MATER,2018,1616-301X,NaN,"CHEMISTRY, MULTIDISCIPLINARY",95431
399,400,ADV FUNCT MATER,2018,1616-301X,NaN,"PHYSICS, APPLIED",95431
400,401,ADV FUNCT MATER,2018,1616-301X,NaN,"CHEMISTRY, PHYSICAL",95431
401,402,ADV FUNCT MATER,2018,1616-301X,NaN,"PHYSICS, CONDENSED MATTER",95431
402,403,ADV FUNCT MATER,2018,1616-301X,NaN,NANOSCIENCE & NANOTECHNOLOGY,95431
418,419,ADV MATER,2018,0935-9648,NaN,"CHEMISTRY, MULTIDISCIPLINARY",229186
419,420,ADV MATER,2018,0935-9648,NaN,"MATERIALS SCIENCE, MULTIDISCIPLINARY",229186
420,421,ADV MATER,2018,0935-9648,NaN,"PHYSICS, CONDENSED MATTER",229186
421,422,ADV MATER,2018,0935-9648,NaN,NANOSCIENCE & NANOTECHNOLOGY,229186


In [ ]:
df_test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2018/Impact_factor_2018_2.csv')

In [ ]:



  # https://stackoverflow.com/questions/12851791/removing-numbers-from-string

  ## NOTA: para corregir otras columnas a las que les pasa lo mismo, se debe
  ## implementar un IF que discrimine por valores nulo esa columna. 

# extracted_text = df_test.iloc[:, 0].map(lambda x: ''.join([i for i in x if not i.isdigit() & pd.isna(i) != False ]))
  # extracted_text
extracted_text = ''
for j in np.arange(998,df_test.shape[0]):

  if pd.isna(df_test[df_test.columns[0]][j]) != True:
    # print(df_test[df_test.columns[0]][j])
    # extracted_text.append( ''.join([i for i in df_test[df_test.columns[0]][j]  if not i.isdigit()]))
    extracted_text = ''.join([i for i in df_test[df_test.columns[0]][j]  if not i.isdigit()])
    
    removed_text = ''.join([i for i in df_test[df_test.columns[0]][j] if not i.isalpha()]).replace([['-','+']],[['','']]).strip()
    
    if extracted_text != '':
      
      df_test.iloc[j,1] = extracted_text[1:]# [1:] es para quitarle el espacio inicial
      df_test.iloc[j,0] = removed_text
  else:
    print('Original data ',df_test[df_test.columns[1]][j])
    removed_text = ''.join([i for i in df_test[df_test.columns[1]][j] if not i.isalpha()]).replace([['-','+']],[['','']]).strip() # eliminar '-' y elimiinar espacios

    extracted_text = ''.join([i for i in df_test[df_test.columns[1]][j] if not i.isdigit()])
    # print(j)
    print('Extracted_text',extracted_text)
    print('Removed text',removed_text)
    
    df_test.iloc[j,1] = extracted_text[1:]
    print('Updated value', df_test[df_test.columns[1]][j], '\n')
    df_test.iloc[j,0]= removed_text 



print(df_test.iloc[13810:13816,:])
# print(df_test.iloc[3136:,:])
df_test.tail()



NameError: ignored

In [ ]:
df_test = df_1.copy()
df_test = extract_index_text(999,df_test)

categories = df_test['CATEGORY_DESCRIPTION'].unique() # también guarda los nan
categories = list(filter(lambda x: pd.isna(x) is not True, categories)) # eliminamos los nan de categories
df_test = replace_nan_values_in_category_description(df_test, categories)
df_test.isnull().sum()/df_test.shape[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Unnamed: 0              0.0
Title20                 0.0
Year                    0.0
ISSN                    0.0
Categories              0.0
CATEGORY_DESCRIPTION    0.0
TOT_CITES               0.0
dtype: float64

In [ ]:
save_df_to_csv(df_test, filename_route_out='/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2018',
               year = 'prueba', i=0)
df_test_2 = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2018/Impact_factor_prueba_0.csv')
df_test_2.isnull().sum()/df_test_2.shape[0]

CSV saved!


Unnamed: 0              0.000000
Title20                 0.000069
Year                    0.000000
ISSN                    0.000000
Categories              0.000000
CATEGORY_DESCRIPTION    0.000000
TOT_CITES               0.000000
dtype: float64

In [ ]:
df_test_2[df_test_2['Title20'].isnull()]

,Unnamed: 0,Title20,Year,ISSN,Categories,CATEGORY_DESCRIPTION,TOT_CITES
5102,5103 3,NaN,2018,2160-1836,KM,GENETICS & HEREDITY,5441


In [ ]:
df_1.iloc[5102,:]

Unnamed: 0                           5103 3
Title20                 G-GENES GENOM GENET
Year                                   2018
ISSN                              2160-1836
Categories                               KM
CATEGORY_DESCRIPTION    GENETICS & HEREDITY
TOT_CITES                              5441
Name: 5102, dtype: object

In [ ]:
df_test[df_test['CATEGORY_DESCRIPTION'].isnull()]

,Unnamed: 0,Title20,Year,ISSN,Categories,CATEGORY_DESCRIPTION,TOT_CITES
397,398,ADV FUNCT MATER,2018,1616-301X,"DY, PM, EI, UK, UB, NS MATERIALS SCIENCE, MULT...",NaN,95431
398,399,ADV FUNCT MATER,2018,1616-301X,"DY, PM, EI, UK, UB, NS CHEMISTRY, MULTIDISCIPL...",NaN,95431
399,400,ADV FUNCT MATER,2018,1616-301X,"DY, PM, EI, UK, UB, NS PHYSICS, APPLIED",NaN,95431
400,401,ADV FUNCT MATER,2018,1616-301X,"DY, PM, EI, UK, UB, NS CHEMISTRY, PHYSICAL",NaN,95431
401,402,ADV FUNCT MATER,2018,1616-301X,"DY, PM, EI, UK, UB, NS PHYSICS, CONDENSED MATTER",NaN,95431
402,403,ADV FUNCT MATER,2018,1616-301X,"DY, PM, EI, UK, UB, NS NANOSCIENCE & NANOTECHN...",NaN,95431
418,419,ADV MATER,2018,0935-9648,"UB, UK, EI, DY, PM, NS CHEMISTRY, MULTIDISCIPL...",NaN,229186
419,420,ADV MATER,2018,0935-9648,"UB, UK, EI, DY, PM, NS MATERIALS SCIENCE, MULT...",NaN,229186
420,421,ADV MATER,2018,0935-9648,"UB, UK, EI, DY, PM, NS PHYSICS, CONDENSED MATTER",NaN,229186
421,422,ADV MATER,2018,0935-9648,"UB, UK, EI, DY, PM, NS NANOSCIENCE & NANOTECHN...",NaN,229186


In [ ]:
categories = df_test['CATEGORY_DESCRIPTION'].unique() # también guarda los nan
categories = list(filter(lambda x: pd.isna(x) is not True, categories)) # eliminamos los nan de categories

df_test_2 = replace_nan_values_in_category_description(df_test, categories)


Found!
Category: MATERIALS SCIENCE, MULTIDISCIPLINARY
Updated value: MATERIALS SCIENCE, MULTIDISCIPLINARY
Index_text 397

Found!
Category: MATERIALS SCIENCE, MULTIDISCIPLINARY
Updated value: MATERIALS SCIENCE, MULTIDISCIPLINARY
Index_text 419

Found!
Category: MATERIALS SCIENCE, MULTIDISCIPLINARY
Updated value: MATERIALS SCIENCE, MULTIDISCIPLINARY
Index_text 10795

Found!
Category: MATERIALS SCIENCE, MULTIDISCIPLINARY
Updated value: MATERIALS SCIENCE, MULTIDISCIPLINARY
Index_text 13319

Found!
Category: MATHEMATICS
Updated value: MATHEMATICS
Index_text 2723

Found!
Category: MATHEMATICS
Updated value: MATHEMATICS
Index_text 7603

Found!
Category: HEALTH CARE SCIENCES & SERVICES
Updated value: HEALTH CARE SCIENCES & SERVICES
Index_text 3830

Found!
Category: HEALTH CARE SCIENCES & SERVICES
Updated value: HEALTH CARE SCIENCES & SERVICES
Index_text 4431

Found!
Category: CHEMISTRY, MULTIDISCIPLINARY
Updated value: CHEMISTRY, MULTIDISCIPLINARY
Index_text 398

Found!
Category: CHEMISTRY, M

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Updated value: ONCOLOGY
Index_text 4433

Found!
Category: NURSING
Updated value: NURSING
Index_text 4432

Found!
Category: REHABILITATION
Updated value: REHABILITATION
Index_text 3831

Found!
Category: REHABILITATION
Updated value: REHABILITATION
Index_text 4434

Found!
Category: STATISTICS & PROBABILITY
Updated value: STATISTICS & PROBABILITY
Index_text 2719

Found!
Category: STATISTICS & PROBABILITY
Updated value: STATISTICS & PROBABILITY
Index_text 7605

Found!
Category: PUBLIC, ENVIRONMENTAL & OCCUPATIONAL HEALTH
Updated value: PUBLIC, ENVIRONMENTAL & OCCUPATIONAL HEALTH
Index_text 3829

Found!
Category: AUTOMATION & CONTROL SYSTEMS
Updated value: AUTOMATION & CONTROL SYSTEMS
Index_text 2724

Found!
Category: AUTOMATION & CONTROL SYSTEMS
Updated value: AUTOMATION & CONTROL SYSTEMS
Index_text 7601


In [ ]:
df_test_2.isnull().sum()/df_test_2.shape[0]

Unnamed: 0              0.0
Title20                 0.0
Year                    0.0
ISSN                    0.0
Categories              0.0
CATEGORY_DESCRIPTION    0.0
TOT_CITES               0.0
dtype: float64

In [ ]:
df_test_2 = df_1.copy()

extracted_text = []
extracted_text = ''.join([i for i in df_test_2[df_test_2.columns[1]][3136] if not i.isdigit()])
extracted_text[1:]

'COMMUN NUMBER THEORY'

In [ ]:
removed_text = []
removed_text = ''.join([i for i in df_test_2[df_test_2.columns[1]][3136] if not i.isalpha()])
removed_text.strip()

'3137'

In [ ]:
df_test.isnull().sum()/df_test.shape[0]

Unnamed: 0              0.000000
Title20                 0.000000
Year                    0.000000
ISSN                    0.000000
Categories              0.000000
CATEGORY_DESCRIPTION    0.003105
TOT_CITES               0.000000
dtype: float64

In [ ]:
df_test[df_test[df_test.columns[0]].isnull()]

extracted_text = ''.join([i for i in df_test[df_test.columns[1]][3136]  if not i.isdigit()])
print(extracted_text[1:])
removed_text = ''.join([i for i in df_test[df_test.columns[1]][3136] if not i.isalpha()])
print(removed_text)

df_test[df_test.columns[1]][3136] = extracted_text[1:]
df_test[df_test.columns[0]][3136] = removed_text.strip()

df_test.iloc[3136, :]

COMMUN NUMBER THEORY
3137   


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Unnamed: 0                              3137
Title20                 COMMUN NUMBER THEORY
Year                                    2018
ISSN                               1931-4523
Categories                        UR, PQ, PN
CATEGORY_DESCRIPTION    MATHEMATICS, APPLIED
TOT_CITES                                435
Name: 3136, dtype: object

In [ ]:
df_test = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020/Impact_factor_2020.csv')
df_test.isnull().sum()/df_test.shape[0]
# df_test = extract_index_text(999, df_test)
# df_test.isnull().sum()/df_test.shape[0]
# df_test.dtypes

Unnamed: 0              0.000000
Index                   0.001454
Title                   0.647852
Year                    0.000000
ISSN                    0.000000
Categories              0.000000
Category_Description    0.003305
Total_cites             0.000000
dtype: float64

In [ ]:
df_test[df_test['Index'].isnull()]

,Unnamed: 0,Index,Title,Year,ISSN,Categories,Category_Description,Total_cites
3249,3249,NaN,3250 COMMUN NUMBER THEORY,2020,1931-4523,"UR, PQ, PN","MATHEMATICS, APPLIED",465
3250,3250,NaN,3251 COMMUN NUMBER THEORY,2020,1931-4523,"UR, PQ, PN","PHYSICS, MATHEMATICAL",465
3251,3251,NaN,3252 COMMUN NUMBER THEORY,2020,1931-4523,"UR, PQ, PN",MATHEMATICS,465
4729,4729,NaN,4730 EUR NEUROPSYCHOPHARM,2020,0924-977X,"RU, VE, TU, RT",PHARMACOLOGY & PHARMACY,8999
4730,4730,NaN,4731 EUR NEUROPSYCHOPHARM,2020,0924-977X,"RU, VE, TU, RT",PSYCHIATRY,8999
4731,4731,NaN,4732 EUR NEUROPSYCHOPHARM,2020,0924-977X,"RU, VE, TU, RT",NEUROSCIENCES,8999
4732,4732,NaN,4733 EUR NEUROPSYCHOPHARM,2020,0924-977X,"RU, VE, TU, RT",CLINICAL NEUROLOGY,8999
5354,5354,NaN,5355 GENE CHROMOSOME CANC,2020,1045-2257,"KM, DM",ONCOLOGY,6108
5355,5355,NaN,5356 GENE CHROMOSOME CANC,2020,1045-2257,"KM, DM",GENETICS & HEREDITY,6108
5519,5519,NaN,5520 GEOTEXT GEOMEMBRANES,2020,0266-1144,"LE, IX","ENGINEERING, GEOLOGICAL",5999


In [ ]:
extracted_text = df_test.iloc[1001, 1].map(lambda x: ''.join([i for i in x if not i.isdigit()]))
# extracted_text

# # eliminamos texto colado en la columna índice
# removed_text = df_test.iloc[:, 0].map(lambda x: ''.join([i for i in x if not i.isalpha()]))
# # removed_text

# # reemplazamoos los valores correspondientes a partir del 999 en el index del dataframe(no la columna 'Index').
# # cuando se reemplacen en los df donde no empiece en 999, ajustarlo
# starter_row_index = 999
# for i in np.arange(starter_row_index,df_test.shape[0]):
#   df['Title20'][i] = extracted_text[i]
#   df['Index'][i] = removed_text[i]

AttributeError: ignored

In [ ]:
for x in df_test.iloc[:,0]:
  extracted_text = ''.join([i for i in x if not i.isdigit()])

TypeError: ignored

In [ ]:
df_test.iloc[:, 4]

0       2053-1583
1       2190-572X
2       2329-7662
3       2329-7662
4       1619-4500
          ...    
7560    0146-4760
7561    0146-4760
7562    0021-8782
7563    0003-2778
7564    0913-8668
Name: ISSN, Length: 7565, dtype: object

In [ ]:
df_test_2 = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020/Impact_factor_Prueba_0.csv')
df_test_2.iloc[998:1005, 1].map(lambda x: ''.join([i for i in x if not i.isdigit()]))

TypeError: ignored

In [ ]:
df_test_2.iloc[990:998,1] # no da fallo porque no está pillando los nan. desde que pilla los nan, me
# da error de que float o int no es interable, por que for i in nan no es iterable (lista o string)

990       ANN GLOB HEALTH
991           ANN HEMATOL
992    ANN HENRI POINCARE
993    ANN HENRI POINCARE
994    ANN HENRI POINCARE
995           ANN HEPATOL
996          ANN HUM BIOL
997          ANN HUM BIOL
Name: Title, dtype: object

In [ ]:
# https://stackoverflow.com/questions/14984119/python-pandas-remove-duplicate-columns
# https://www.statology.orag/pandas-merge-on-index/
# https://www.w3schools.com/sql/sql_join.asp

#concatenamos las 3 tablas
a = pd.concat([df_1, df_2, df_3], axis = 1)
# eliminamos las columnas duplicadas
a = a.loc[:,~a.columns.duplicated()]
a.head(10)

,Unnamed: 0,Title20,Year,ISSN,Categories,CATEGORY_DESCRIPTION,TOT_CITES,IMPACT_FACTOR,Number of Journals in Category,ARTL_INFLUENCE,CATEGORY_RANKING,IMMEDIACY_INDEX,CITED_HALF_LIFE,5YR_IMPACT_FACTOR,EIGENFACTOR,Rank by Impact Factor,QUARTILE_RANK
0,1,2D MATER,2020,2053-1583,PM,"MATERIALS SCIENCE, MULTIDISCIPLINARY",9244,"7,103",N/D,"2,167",Q1,"2,404","3,5","8,161","0,02239",N/D,72/334
1,2,3 BIOTECH,2020,2190-572X,DB,BIOTECHNOLOGY & APPLIED MICROBIOLOGY,6625,"2,406",N/D,"0,469",Q3,"0,542","3,6","3,203","0,00822",N/D,113/159
2,3,3D PRINT ADDIT MANUF,2020,2329-7662,"IK, PM","MATERIALS SCIENCE, MULTIDISCIPLINARY",1123,"5,449",N/D,"1,068",Q2,"1,051",4,"5,48","0,00167",N/D,96/334
3,4,3D PRINT ADDIT MANUF,2020,2329-7662,"IK, PM","ENGINEERING, MANUFACTURING",1123,"5,449",N/D,"1,068",Q2,"1,051",4,"5,48","0,00167",N/D,14/50
4,5,4OR-Q J OPER RES,2020,1619-4500,PE,OPERATIONS RESEARCH & MANAGEMENT SCIENCE,678,"1,545",N/D,"0,784",Q3,"0,477","9,6","2,176","0,00068",N/D,63/84
5,6,AAPG BULL,2020,0149-1423,LE,"GEOSCIENCES, MULTIDISCIPLINARY",18433,"3,529",N/D,"1,09",Q2,"1,233","18,6","4,438","0,00498",N/D,69/200
6,7,AAPS J,2020,1550-7416,TU,PHARMACOLOGY & PHARMACY,7617,"4,009",N/D,"1,009",Q2,"0,913","7,1","4,743","0,00671",N/D,110/276
7,8,AAPS PHARMSCITECH,2020,1530-9932,TU,PHARMACOLOGY & PHARMACY,10553,"3,246",N/D,"0,462",Q3,"0,782","6,7","3,473","0,00624",N/D,149/276
8,9,AATCC J RES,2020,2330-5517,QJ,"MATERIALS SCIENCE, TEXTILES",193,"1,127",N/D,"0,169",Q3,"0,176","3,6","1,029","0,00023",N/D,14/25
9,10,AATCC REV,2020,1532-8813,"DW, II, QJ","ENGINEERING, CHEMICAL",173,"0,125",N/D,"0,063",Q4,0,15,"0,308","0,00002",N/D,142/143


In [ ]:
df_1[df_1['CATEGORY_DESCRIPTION'].isnull()]

,Unnamed: 0,Title20,Year,ISSN,Categories,CATEGORY_DESCRIPTION,TOT_CITES
419,420,ADV FUNCT MATER,2020,1616-301X,"DY, PM, EI, UK, UB, NS NANOSCIENCE & NANOTECHN...",NaN,151020
420,421,ADV FUNCT MATER,2020,1616-301X,"DY, PM, EI, UK, UB, NS PHYSICS, CONDENSED MATTER",NaN,151020
421,422,ADV FUNCT MATER,2020,1616-301X,"DY, PM, EI, UK, UB, NS PHYSICS, APPLIED",NaN,151020
422,423,ADV FUNCT MATER,2020,1616-301X,"DY, PM, EI, UK, UB, NS CHEMISTRY, PHYSICAL",NaN,151020
423,424,ADV FUNCT MATER,2020,1616-301X,"DY, PM, EI, UK, UB, NS MATERIALS SCIENCE, MULT...",NaN,151020
424,425,ADV FUNCT MATER,2020,1616-301X,"DY, PM, EI, UK, UB, NS CHEMISTRY, MULTIDISCIPL...",NaN,151020
441,442,ADV MATER,2020,0935-9648,"UB, UK, EI, DY, PM, NS CHEMISTRY, MULTIDISCIPL...",NaN,320972
442,443,ADV MATER,2020,0935-9648,"UB, UK, EI, DY, PM, NS NANOSCIENCE & NANOTECHN...",NaN,320972
443,444,ADV MATER,2020,0935-9648,"UB, UK, EI, DY, PM, NS CHEMISTRY, PHYSICAL",NaN,320972
444,445,ADV MATER,2020,0935-9648,"UB, UK, EI, DY, PM, NS MATERIALS SCIENCE, MULT...",NaN,320972


In [ ]:
df_1[df_1['CATEGORY_DESCRIPTION'].isnull()]['Categories']

419      DY, PM, EI, UK, UB, NS NANOSCIENCE & NANOTECHN...
420       DY, PM, EI, UK, UB, NS PHYSICS, CONDENSED MATTER
421                DY, PM, EI, UK, UB, NS PHYSICS, APPLIED
422             DY, PM, EI, UK, UB, NS CHEMISTRY, PHYSICAL
423      DY, PM, EI, UK, UB, NS MATERIALS SCIENCE, MULT...
424      DY, PM, EI, UK, UB, NS CHEMISTRY, MULTIDISCIPL...
441      UB, UK, EI, DY, PM, NS CHEMISTRY, MULTIDISCIPL...
442      UB, UK, EI, DY, PM, NS NANOSCIENCE & NANOTECHN...
443             UB, UK, EI, DY, PM, NS CHEMISTRY, PHYSICAL
444      UB, UK, EI, DY, PM, NS MATERIALS SCIENCE, MULT...
445                UB, UK, EI, DY, PM, NS PHYSICS, APPLIED
446       UB, UK, EI, DY, PM, NS PHYSICS, CONDENSED MATTER
2813          AC, OA, EA, EP, XY, PO CHEMISTRY, ANALYTICAL
2814       AC, OA, EA, EP, XY, PO STATISTICS & PROBABILITY
2815     AC, OA, EA, EP, XY, PO INSTRUMENTS & INSTRUMEN...
2816     AC, OA, EA, EP, XY, PO COMPUTER SCIENCE, ARTIF...
2817     AC, OA, EA, EP, XY, PO AUTOMATION & CONTROL SY.

In [ ]:
categories = df_1['CATEGORY_DESCRIPTION'].unique() # también guarda los nan
test = df_1[df_1['CATEGORY_DESCRIPTION'].isnull()]['Categories'].reset_index(drop = True)

In [ ]:
len(categories)

179

In [ ]:
categories = df_1['CATEGORY_DESCRIPTION'].unique() # también guarda los nan
categories = list(filter(lambda x: pd.isna(x) is not True, categories)) # eliminamos los nan de categories
test = df_1[df_1['CATEGORY_DESCRIPTION'].isnull()]['Categories'].reset_index(drop = True)

df_1_copy = df_1.copy()


for i in np.arange(len(categories)):
  for j in np.arange(len(test)):  
    if categories[i] in test[j]:
      print('\nFound!')
      print('Category:', categories[i])
      print('Text',test[j])
      print('Index_text', j)



        


Found!
Category: MATERIALS SCIENCE, MULTIDISCIPLINARY
Text DY, PM, EI, UK, UB, NS MATERIALS SCIENCE, MULTIDISCIPLINARY
Index_text 4

Found!
Category: MATERIALS SCIENCE, MULTIDISCIPLINARY
Text UB, UK, EI, DY, PM, NS MATERIALS SCIENCE, MULTIDISCIPLINARY
Index_text 9

Found!
Category: MATERIALS SCIENCE, MULTIDISCIPLINARY
Text DY, NS, PM, UK, UB, EI MATERIALS SCIENCE, MULTIDISCIPLINARY
Index_text 33

Found!
Category: MATERIALS SCIENCE, MULTIDISCIPLINARY
Text UB, UK, EI, DY, NS, PM MATERIALS SCIENCE, MULTIDISCIPLINARY
Index_text 40

Found!
Category: MATHEMATICS
Text AC, OA, EA, EP, XY, PO MATHEMATICS, INTERDISCIPLINARY APPLICATIONS
Index_text 17

Found!
Category: MATHEMATICS
Text EP, PO, OA, AC, XY, EA MATHEMATICS, INTERDISCIPLINARY APPLICATIONS
Index_text 25

Found!
Category: HEALTH CARE SCIENCES & SERVICES
Text WC, HL, NE, WC, NE, LQHEALTH CARE SCIENCES & SERVICES
Index_text 19

Found!
Category: HEALTH CARE SCIENCES & SERVICES
Text WC, HL, DM, RZ, RZ, WCHEALTH CARE SCIENCES & SERVICES
In

In [ ]:
def replace_nan_values_in_category_description(df):
  df_1_copy=df.copy()

  for i in np.arange(len(categories)):
    for index in df_1_copy[df_1_copy['CATEGORY_DESCRIPTION'].isnull()].index:
      if categories[i] in df_1_copy['Categories'][index]:
        print('\nFound!')
        print('Category:', categories[i])
        df_1_copy['CATEGORY_DESCRIPTION'][index] = categories[i]
        print('Updated value:', df_1_copy['CATEGORY_DESCRIPTION'][index] )
        print('Index_text', index)
        df_1_copy['Categories'][index] = df_1_copy['Categories'][index].replace(categories[i],'')

  return df




Found!
Category: MATERIALS SCIENCE, MULTIDISCIPLINARY
Updated value: MATERIALS SCIENCE, MULTIDISCIPLINARY
Index_text 423

Found!
Category: MATERIALS SCIENCE, MULTIDISCIPLINARY
Updated value: MATERIALS SCIENCE, MULTIDISCIPLINARY
Index_text 444

Found!
Category: MATERIALS SCIENCE, MULTIDISCIPLINARY
Updated value: MATERIALS SCIENCE, MULTIDISCIPLINARY
Index_text 11237

Found!
Category: MATERIALS SCIENCE, MULTIDISCIPLINARY
Updated value: MATERIALS SCIENCE, MULTIDISCIPLINARY
Index_text 13865

Found!
Category: MATHEMATICS
Updated value: MATHEMATICS
Index_text 2818

Found!
Category: MATHEMATICS
Updated value: MATHEMATICS
Index_text 7908

Found!
Category: HEALTH CARE SCIENCES & SERVICES
Updated value: HEALTH CARE SCIENCES & SERVICES
Index_text 3974

Found!
Category: HEALTH CARE SCIENCES & SERVICES
Updated value: HEALTH CARE SCIENCES & SERVICES
Index_text 4610

Found!
Category: CHEMISTRY, MULTIDISCIPLINARY
Updated value: CHEMISTRY, MULTIDISCIPLINARY
Index_text 424

Found!
Category: CHEMISTRY, M

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


 CHEMISTRY, ANALYTICAL
Index_text 2813

Found!
Category: CHEMISTRY, ANALYTICAL
Updated value: CHEMISTRY, ANALYTICAL
Index_text 7909

Found!
Category: INSTRUMENTS & INSTRUMENTATION
Updated value: INSTRUMENTS & INSTRUMENTATION
Index_text 2815

Found!
Category: INSTRUMENTS & INSTRUMENTATION
Updated value: INSTRUMENTS & INSTRUMENTATION
Index_text 7912

Found!
Category: NANOSCIENCE & NANOTECHNOLOGY
Updated value: NANOSCIENCE & NANOTECHNOLOGY
Index_text 419

Found!
Category: NANOSCIENCE & NANOTECHNOLOGY
Updated value: NANOSCIENCE & NANOTECHNOLOGY
Index_text 442

Found!
Category: NANOSCIENCE & NANOTECHNOLOGY
Updated value: NANOSCIENCE & NANOTECHNOLOGY
Index_text 11239

Found!
Category: NANOSCIENCE & NANOTECHNOLOGY
Updated value: NANOSCIENCE & NANOTECHNOLOGY
Index_text 13864

Found!
Category: COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE
Updated value: COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE
Index_text 2816

Found!
Category: COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE
Updated value: COMPUTER SCIEN

In [ ]:
df_1_copy.isnull().sum()/df_1_copy.shape[0]

Unnamed: 0              0.000729
Title20                 0.662339
Year                    0.000000
ISSN                    0.000000
Categories              0.000000
CATEGORY_DESCRIPTION    0.000000
TOT_CITES               0.000000
dtype: float64

In [ ]:
df_1_copy[df_1_copy['CATEGORY_DESCRIPTION'].isnull()]

,Unnamed: 0,Title20,Year,ISSN,Categories,CATEGORY_DESCRIPTION,TOT_CITES


In [ ]:
df_1_copy['Categories'][419]

'DY, PM, EI, UK, UB, NS '

In [ ]:
test[0] # NANOSCIENCE & NANOTECHNOLOGY
if categories[123] in 'EP, PO, OA, AC, XY, EA MATHEMATICS, INTERDISCIPLINARY APPLICATIONS':
  print('ok')

ok


In [ ]:
if pd.isna(categories[126]) == True:
  print('ok')

ok


In [ ]:
categories = list(filter(lambda x: pd.isna(x) is not True, categories))
len(categories)
# categories

178

In [ ]:
# df_prueba = pd.DataFrame({'Index':df[df.columns[0]]})

df_prueba = pd.DataFrame({'Index':df[df.columns[0]],'Title':df[df.columns[1]],
                          'Year':df[df.columns[2]], 'ISSN':df[df.columns[3]],
                          'Categories': df[df.columns[4]],
                          'Category_Description':df[df.columns[5]], 
                          'Total_cites':df[df.columns[6]]} )

df_prueba.head()

save_df_to_csv(df_prueba,'/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020',
               '2020')
test = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020/Impact_factor_2020.csv')
test.isnull().sum()/test.shape[0]


CSV saved!


Unnamed: 0              0.000000
Index                   0.001454
Title                   0.647852
Year                    0.000000
ISSN                    0.000000
Categories              0.000000
Category_Description    0.003305
Total_cites             0.000000
dtype: float64

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020/prueba.csv')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020/prueba.csv')
df.head(10)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 1,Title20,Year,ISSN,Categories,CATEGORY_DESCRIPTION,TOT_CITES
0,0,0,1,2D MATER,2020,2053-1583,PM,"MATERIALS SCIENCE, MULTIDISCIPLINARY",9244
1,1,1,2,3 BIOTECH,2020,2190-572X,DB,BIOTECHNOLOGY & APPLIED MICROBIOLOGY,6625
2,2,2,3,3D PRINT ADDIT MANUF,2020,2329-7662,"IK, PM","MATERIALS SCIENCE, MULTIDISCIPLINARY",1123
3,3,3,4,3D PRINT ADDIT MANUF,2020,2329-7662,"IK, PM","ENGINEERING, MANUFACTURING",1123
4,4,4,5,4OR-Q J OPER RES,2020,1619-4500,PE,OPERATIONS RESEARCH & MANAGEMENT SCIENCE,678
5,5,5,6,AAPG BULL,2020,0149-1423,LE,"GEOSCIENCES, MULTIDISCIPLINARY",18433
6,6,6,7,AAPS J,2020,1550-7416,TU,PHARMACOLOGY & PHARMACY,7617
7,7,7,8,AAPS PHARMSCITECH,2020,1530-9932,TU,PHARMACOLOGY & PHARMACY,10553
8,8,8,9,AATCC J RES,2020,2330-5517,QJ,"MATERIALS SCIENCE, TEXTILES",193
9,9,9,10,AATCC REV,2020,1532-8813,"DW, II, QJ","ENGINEERING, CHEMICAL",173


In [ ]:
# df = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020/Impact_factor_2020.csv')
df.isnull().sum()/df.shape[0]

0
                        0.0
Title20                 0.0
Year                    0.0
ISSN                    0.0
Categories              0.0
CATEGORY_DESCRIPTION    0.0
TOT_CITES               0.0
dtype: float64

In [ ]:
df.columns=columns1
df.head(5)
df.isnull().sum()/df.shape[0]

save_df_to_csv(df,'/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020',
               '2020')
test = read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020/Impact_factor_2020.csv')
test.isnull().sum()/df.shape[0]

CSV saved!


Unnamed: 0              0.000000
Index                   0.001454
Title                   0.647852
Year                    0.000000
ISSN                    0.000000
Categories              0.000000
Category_Description    0.003305
Total_cites             0.000000
dtype: float64

In [ ]:
read_csv('/content/drive/MyDrive/Colab Notebooks/PDFs_correctos_por_año/Impact_factor_2020/Impact_factor_2020.csv')

FileNotFoundError: ignored

In [ ]:
print("Total tables extracted:", table.n)


df.columns = table[0].df.iloc[0,:]
df

Total tables extracted: 291


,,Title20,Year,ISSN,Categories,CATEGORY_DESCRIPTION,TOT_CITES
0,1,2D MATER,2020,2053-1583,PM,"MATERIALS SCIENCE, MULTIDISCIPLINARY",9244
1,2,3 BIOTECH,2020,2190-572X,DB,BIOTECHNOLOGY & APPLIED MICROBIOLOGY,6625
2,3,3D PRINT ADDIT MANUF,2020,2329-7662,"IK, PM","MATERIALS SCIENCE, MULTIDISCIPLINARY",1123
3,4,3D PRINT ADDIT MANUF,2020,2329-7662,"IK, PM","ENGINEERING, MANUFACTURING",1123
4,5,4OR-Q J OPER RES,2020,1619-4500,PE,OPERATIONS RESEARCH & MANAGEMENT SCIENCE,678
...,...,...,...,...,...,...,...
7560,7561,J ANAL TOXICOL,2020,0146-4760,"YO, EA",TOXICOLOGY,4796
7561,7562,J ANAL TOXICOL,2020,0146-4760,"YO, EA","CHEMISTRY, ANALYTICAL",4796
7562,7563,J ANAT,2020,0021-8782,AY,ANATOMY & MORPHOLOGY,11655
7563,7564,J ANAT SOC INDIA,2020,0003-2778,AY,ANATOMY & MORPHOLOGY,243


In [ ]:
# def read_and_format_columns(tables, columns = []):

#     #Leemos la tabla que previamente hemos cargado con la librería Camelot
#     df = pd.DataFrame(tables.df)
#     # Eliminamos la primera fila, que contiene el nombre de las columnas
#     df = df.drop(0, axis = 0)
#     #cambiamos el nombre de las columnas a las que hemos establecido
#     df.columns = [columns]
# #     test.head()
#     #devolvemos el dataframe formateado
#     return df

def read_and_format_columns_from_tables(tables, columns = []):
    #Leemos la tabla que previamente hemos cargado con la librería Camelot
    df = pd.DataFrame(tables.df)
    # Eliminamos la primera fila, que contiene el nombre de las columnas
    df = df.drop(0, axis = 0)
    #cambiamos el nombre de las columnas a las que hemos establecido
    df.columns = columns
#     test.head()
    #devolvemos el dataframe formateado
    return df

# read_and_format_columns(tables[1], columns = ['Index','Title20', 'Year', 'Country', 'ISSN', 'Language', 'Categories'])

In [ ]:
# def df_joined_rows(table, columns):
#     '''
# 1. Pasamos las tablas cargadas previamente con la librería camelot
# 2. Usamos la función read_and_format_columns para convertir esas tablas a Dataframe
# con las columnas indicadas
# 3. Concatenamos todos esos Dataframes en un dataframe final
#     '''
#     for i in np.arange(0,table.n):
#       if i == 0:
#           df_rows_joined = pd.concat([read_and_format_columns(table[i], columns), read_and_format_columns(table[i+1], columns)])
#       else:
#         try:
#           df_rows_joined = pd.concat([df_rows_joined, read_and_format_columns(table[i+1], columns)])
#         except:
#           pass

#     # Devuelve en un dataframe todas las tablas leídas del pdf y concatenadas una
#     # debajo de otra
#     return df_rows_joined.reset_index(drop= True)

## hacemos una modificacion para no pasar columns como una variable definida
def df_joined_rows(table):
    '''
1. Pasamos las tablas cargadas previamente con la librería camelot
2. Usamos la función read_and_format_columns_from_tables para convertir esas tablas a Dataframe
con las columnas indicadas
3. Concatenamos todos esos Dataframes en un dataframe final
    '''
    columns = table[0].df.iloc[0,:]
    for i in np.arange(0,table.n):
      if i == 0:
          df_rows_joined = pd.concat([read_and_format_columns_from_tables(table[i], columns), read_and_format_columns_from_tables(table[i+1], columns)])
      else:
        try:
          # indicamos el +1 dentro del try except para que concatene la siguiente tabla
          # a la actual, cuando se produzca el out of bound, siga ejecutándose y 
          # termine
          df_rows_joined = pd.concat([df_rows_joined, read_and_format_columns_from_tables(table[i+1], columns)])
        except:
          pass

    # Devuelve en un dataframe todas las tablas leídas del pdf y concatenadas una
    # debajo de otra
    return df_rows_joined.reset_index(drop= True)

In [ ]:
## usamos .map() para pd.Series y .apply() para pd.Dataframe

## Estos comandos sólo se hacen una vez, ya que una vez aplicados no tiene 
## sentido el cambio y puede incluir datos erróneos

## Esta función no hace falta aplicarla si las columnas del dataframe no las 
## renombramos. Si por el contrario, le asignamos otros nombres como hacíamos en 
## anteriores pruebas, algunos datos y columnas se solapaban.

def extract_index_text(starter_row_index, df):
  # https://stackoverflow.com/questions/12851791/removing-numbers-from-string

  ## NOTA: para corregir otras columnas a las que les pasa lo mismo, se debe
  ## implementar un IF que discrimine por valores nulo esa columna. 
  
  extracted_text = df.iloc[:, 0].map(lambda x: ''.join([i for i in x if not i.isdigit()]))
  # extracted_text

  # eliminamos texto colado en la columna índice
  removed_text = df.iloc[:, 0].map(lambda x: ''.join([i for i in x if not i.isalpha()]))
  # removed_text

  # reemplazamoos los valores correspondientes a partir del 999 en el index del dataframe(no la columna 'Index').
  # cuando se reemplacen en los df donde no empiece en 999, ajustarlo.
  for i in np.arange(starter_row_index,df.shape[0]):
    df[1][i] = extracted_text[i]
    df[0][i] = removed_text[i]
  
  return df

In [ ]:
df[df.columns[1]]
df.isnull().sum()/df.shape[0]

0
                        0.0
Title20                 0.0
Year                    0.0
ISSN                    0.0
Categories              0.0
CATEGORY_DESCRIPTION    0.0
TOT_CITES               0.0
dtype: float64

In [ ]:
# https://stackoverflow.com/questions/12851791/removing-numbers-from-string

# ver df.iloc[997:1050,:] para ver como lee mal los datos a partir del índice 999
#extraemos el texto colado en la columna índice
extracted_text = df.iloc[:, 0].map(lambda x: ''.join([i for i in x if not i.isdigit()]))
extracted_text

# eliminamos texto colado en la columna índice
removed_text = df.iloc[:, 0].map(lambda x: ''.join([i for i in x if not i.isalpha()]))
removed_text

# reemplazamoos los valores correspondientes a partir del 999 en el index del dataframe(no la columna 'Index')
for i in np.arange(999,df.shape[0]):
  df['Title'][i] = extracted_text[i]
  df['Index'][i] = removed_text[i]


0                            
1                            
2                            
3                            
4                            
                 ...         
10733     AQUAT LIVING RESOUR
10734     AQUAT LIVING RESOUR
10735     AQUAT LIVING RESOUR
10736     AQUAT LIVING RESOUR
10737     AQUAT LIVING RESOUR
Name: Index, Length: 10738, dtype: object

In [ ]:
# !pip install tabula-py
# import tabula